In [1]:
from pyomo.environ import *

In [2]:
def solve_pyomo(para):
    p_ei  = para['pei']
    p_er  = para['per']
    p_ea  = 1-p_ei-p_er
    p_ai  = 0.10714
    p_ar  = 0.04545
    p_irr = 1/8
    alpha  = 0.6
    beta_1 = para['beta1']
    beta_2 = para['beta2']
    cost_tr = 0.0977
    p_id  = 1.1/1000
    p_idd = 0.1221/1000
    p_srr = 0.7/3
    S = 100000
    E = 20
    A = 0
    I = 0
    R = 0
    D = 0
    N = S+E+A+I+R+D
    K = 50
    w_max  = 1.0
    tr_max = 1.0
    v_max = 0.2/14
    M = 3.6e3
    
    m = ConcreteModel()
    m.k = RangeSet(0, K)
    m.t = RangeSet(0, K-1)
    m.N = N
    m.tri = Var(m.k, bounds=(0, 1)) 
    m.wi = Var(m.k, bounds=(0, 1))  
    m.v = Var(m.k, bounds=(0, 1))
    m.aq = Var(m.k, bounds=(0, 1))
    m.iq = Var(m.k, bounds=(0, 1))
    m.x_s = Var(m.k, domain = NonNegativeReals)
    m.x_e = Var(m.k, domain = NonNegativeReals)
    m.x_a = Var(m.k, domain = NonNegativeReals)
    m.x_i = Var(m.k, domain = NonNegativeReals)
    m.x_r = Var(m.k, domain = NonNegativeReals)
    m.x_d = Var(m.k, domain = NonNegativeReals)
    
    m.xs_c = Constraint(m.t, rule=lambda m, k: m.x_s[k+1] == m.x_s[k]
                    -(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                    *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                    -p_srr*m.v[k]*v_max*m.x_s[k])

    m.xe_c = Constraint(m.t, rule=lambda m, k: m.x_e[k+1] == m.x_e[k]
                        +alpha*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        -p_ei*m.x_e[k]
                        -p_er*m.x_e[k]
                        -m.wi[k]*p_ea*m.x_e[k])
                     
    m.xa_c = Constraint(m.t, rule=lambda m, k: m.x_a[k+1] == m.x_a[k]
                        +m.wi[k]*p_ea*m.x_e[k]
                        -p_ai*m.x_a[k]
                        -p_ar*m.x_a[k])
                                        
    m.xi_c = Constraint(m.t, rule=lambda m, k: m.x_i[k+1] == m.x_i[k]
                        +(1-alpha)*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +p_ei*m.x_e[k]
                        +p_ai*m.x_a[k]
                        -m.tri[k]*p_irr*m.x_i[k]
                        -(1-m.tri[k])*m.x_i[k]*p_id
                        -m.tri[k]*m.x_i[k]*p_idd)
                                                                                       
    m.xr_c = Constraint(m.t, rule=lambda m, k: m.x_r[k+1] == m.x_r[k]
                        +m.tri[k]*p_irr*m.x_i[k]
                        +p_er*m.x_e[k]
                        +p_ar*m.x_a[k]
                        +p_srr*m.v[k]*v_max*m.x_s[k])

    m.xd_c = Constraint(m.t, rule=lambda m, k: m.x_d[k+1] == m.x_d[k]
                        +(1-m.tri[k])*m.x_i[k]*p_id
                        +m.tri[k]*m.x_i[k]*p_idd)
    
    m.pc = ConstraintList()
    m.pc.add(m.x_s[0]==S)
    m.pc.add(m.x_e[0]==E)
    m.pc.add(m.x_a[0]==A)
    m.pc.add(m.x_i[0]==I)
    m.pc.add(m.x_r[0]==R)
    m.pc.add(m.x_d[0]==D)

    gamma = 1
    m.sumcost = sum(m.wi[k]*(0.000369*(m.wi[k]*(m.x_s[k]+m.x_a[k]+m.x_e[k]+m.x_r[k])/m.N)
                         *(m.wi[k]*(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])/m.N) + 0.001057)
                         *(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])
                    +0.0977*m.tri[k]*m.x_i[k]+m.x_s[k]
                    *m.v[k]*v_max*0.07 + m.x_i[k]*m.iq[k]*0.02 + m.x_a[k]*m.aq[k]*0.02 for k in m.t) 
    m.budget_c = Constraint(expr = m.sumcost <= M)

    m.suminfected = sum(2.6 * (beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +(m.x_d[k+1] - m.x_d[k]) * 60 for k in m.t)
    m.obj = Objective(expr = m.suminfected, sense = minimize)
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 50000
    try:
        solver.solve(m,tee=True)
    except:
        print()

In [3]:
import pickle
interval = 0.2
para_truth = pickle.load(open('para_truth' + str(round(interval * 100)), 'rb'))

In [4]:
solve_pyomo(para_truth[0])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 6.0125884e+03 9.86e+04 1.79e+03  -4.5 1.75e+03  -1.7 1.37e-05 9.27e-04f  1
 201r 6.0377275e+03 9.86e+04 1.64e+03  -4.5 4.41e+03  -2.2 1.34e-02 1.51e-03f  1
 202r 6.0735916e+03 9.86e+04 1.64e+03  -4.5 9.13e+03  -2.7 1.70e-03 1.01e-03f  1
 203r 6.1295680e+03 9.86e+04 2.13e+03  -4.5 7.88e+02  -1.3 1.91e-06 1.97e-02f  1
 204r 6.1295826e+03 9.86e+04 2.11e+03  -4.5 3.11e+02  -0.9 6.54e-04 1.38e-05f  1
 205r 6.1298368e+03 9.86e+04 2.10e+03  -4.5 8.82e+02  -1.4 1.62e-04 8.13e-05f  1
 206r 6.1524966e+03 9.86e+04 2.10e+03  -4.5 2.39e+03  -1.9 2.61e-03 2.60e-03f  1
 207r 6.1525215e+03 9.86e+04 2.10e+03  -4.5 5.67e+03  -2.4 1.75e-05 1.16e-06f  1
 208r 6.2129321e+03 9.86e+04 2.09e+03  -4.5 1.08e+04  -2.8 4.31e-04 1.39e-03f  1
 209r 6.2414787e+03 9.86e+04 2.11e+03  -4.5 1.60e+04  -3.3 1.15e-03 3.95e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 210r 6.2414823e+03 9.86e+04

 383r 3.0708995e+04 9.18e+04 2.92e+03  -4.5 1.48e+04    -  7.00e-03 3.08e-07f  1
 384r 3.0709252e+04 9.18e+04 3.87e+03  -4.5 1.49e+04    -  1.60e-02 3.73e-06f  1
 385r 3.0764650e+04 9.17e+04 3.82e+03  -4.5 1.48e+04    -  5.38e-07 8.06e-04f  1
 386r 3.0785281e+04 9.17e+04 3.94e+03  -4.5 1.35e+04    -  1.92e-03 4.61e-04f  1
 387r 3.0804058e+04 9.17e+04 3.94e+03  -4.5 1.34e+04    -  4.64e-04 4.20e-04f  1
 388r 3.0820203e+04 9.17e+04 4.32e+03  -4.5 1.34e+04    -  4.86e-03 3.61e-04f  1
 389r 3.0820295e+04 9.17e+04 6.43e+03  -4.5 1.47e+04    -  2.81e-02 1.34e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 390r 3.0893523e+04 9.17e+04 1.05e+04  -4.5 1.47e+04    -  3.55e-02 1.07e-03f  1
 391r 3.0945250e+04 9.17e+04 1.04e+04  -4.5 1.46e+04    -  1.22e-06 7.55e-04f  1
 392r 3.0946465e+04 9.17e+04 1.32e+04  -4.5 1.42e+04    -  3.04e-02 1.77e-05f  1
 393r 3.1040290e+04 9.17e+04 1.31e+04  -4.5 1.42e+04    -  1.06e-06 1.37e-03f  1
 394r 3.1044970e+04 9.17e+04

 547r 6.2124063e+04 7.99e+04 9.99e+02   4.9 0.00e+00   1.3 0.00e+00 2.91e-07R  6
 548r 6.2123354e+04 7.99e+04 9.99e+02   4.9 1.12e+08    -  7.95e-08 9.71e-09f  1
 549r 6.2138140e+04 7.99e+04 9.99e+02   2.8 3.10e+05    -  5.33e-05 2.33e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 550r 6.2191747e+04 7.98e+04 1.31e+04   2.8 3.05e+05    -  6.24e-03 7.31e-05f  1
 551r 6.4808292e+04 7.88e+04 1.40e+04   2.8 2.99e+05    -  4.48e-03 3.93e-03f  1
 552r 6.7108604e+04 7.78e+04 3.35e+04   2.8 3.48e+05    -  9.61e-03 3.20e-03f  1
 553r 7.3920135e+04 7.45e+04 3.46e+04   2.1 4.56e+05    -  8.81e-03 7.65e-03f  1
 554r 7.4153080e+04 7.44e+04 4.38e+04   2.1 7.85e+03  -2.0 2.69e-02 1.25e-02f  1
 555r 7.4415691e+04 7.43e+04 3.63e+04   2.1 4.46e+03  -1.6 1.58e-02 2.80e-02f  1
 556r 7.5247261e+04 7.40e+04 2.53e+04   2.1 7.32e+03  -2.1 3.40e-02 4.68e-02f  1
 557r 7.5535424e+04 7.39e+04 3.02e+04   2.1 3.52e+03  -1.6 4.23e-02 3.11e-02f  1
 558r 7.5718664e+04 7.38e+04

 825  2.2831180e+05 1.17e+02 1.73e+05  -1.0 1.73e+03  -3.1 1.74e-02 9.67e-03h  1
 826  2.2829645e+05 1.17e+02 1.72e+05  -1.0 1.90e+04    -  6.39e-04 3.10e-04f  1
 827  2.2829653e+05 1.17e+02 1.72e+05  -1.0 1.81e+03  -3.6 9.86e-04 1.48e-04h  1
 828  2.2732193e+05 1.14e+02 1.75e+05  -1.0 1.25e+04    -  1.99e-04 2.59e-02f  1
 829  2.2732887e+05 1.13e+02 1.71e+05  -1.0 1.48e+03  -3.1 3.95e-02 8.90e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 830  2.2695547e+05 1.13e+02 1.71e+05  -1.0 9.72e+04    -  2.44e-04 1.86e-03f  1
 831  2.2695716e+05 1.13e+02 1.71e+05  -1.0 1.45e+03  -2.7 1.65e-02 1.08e-03h  1
 832  2.2691909e+05 1.13e+02 1.71e+05  -1.0 2.58e+04    -  1.01e-03 5.82e-04f  1
 833  2.2692256e+05 1.12e+02 1.69e+05  -1.0 1.30e+03  -3.2 8.45e-04 8.71e-03h  1
 834  2.2680598e+05 1.09e+02 1.69e+05  -1.0 2.00e+03  -3.7 5.94e-04 3.06e-02f  1
 835  2.2681018e+05 1.07e+02 1.61e+05  -1.0 1.39e+03  -3.2 6.10e-02 1.30e-02h  1
 836  2.2687483e+05 1.02e+02

...

1578  5.9965196e+03 4.48e+01 1.45e+04  -3.8 5.68e+02  -3.2 3.74e-02 4.12e-04f  1
1579  5.8969833e+03 4.30e+01 8.54e+03  -3.8 1.32e+03  -3.7 5.96e-06 3.84e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1580  5.8921159e+03 4.30e+01 6.78e+03  -3.8 7.31e+02  -3.2 5.99e-02 1.80e-03f  1
1581  5.8665654e+03 4.29e+01 7.09e+03  -3.8 3.91e+03  -3.7 2.86e-06 1.78e-03f  1
1582  5.8336345e+03 4.24e+01 7.31e+03  -3.8 8.36e+02  -3.3 3.42e-03 1.02e-02f  1
1583  5.8335815e+03 4.24e+01 7.17e+03  -3.8 1.07e+04  -3.8 1.88e-04 1.45e-06f  1
1584  5.6091096e+03 3.97e+01 9.28e+03  -3.8 8.99e+02  -3.3 5.97e-04 6.45e-02f  1
1585  5.6060098e+03 3.96e+01 8.67e+03  -3.8 3.77e+02  -2.9 9.08e-02 2.34e-03f  1
1586  5.5450121e+03 3.55e+01 8.47e+03  -3.8 1.64e+02  -2.5 2.78e-02 1.03e-01f  1
1587  5.5289235e+03 3.51e+01 8.33e+03  -3.8 4.16e+02  -3.0 1.87e-02 1.07e-02f  1
1588  5.5287055e+03 3.51e+01 7.47e+03  -3.8 1.23e+03  -3.4 1.74e-02 4.53e-05f  1
1589  5.5025061e+03 3.46e+01

1684  2.2956990e+03 3.21e+00 5.44e+04  -3.8 9.75e+03  -6.8 5.55e-06 2.44e-05f  1
1685  2.2956828e+03 3.21e+00 5.44e+04  -3.8 1.22e+06  -7.3 2.60e-08 3.24e-08f  1
1686  2.2925683e+03 3.21e+00 5.44e+04  -3.8 1.26e+05  -6.9 2.19e-05 2.15e-05f  1
1687  2.2922488e+03 3.21e+00 5.44e+04  -3.8 6.70e+04  -7.3 4.33e-06 1.62e-05f  1
1688  2.2922389e+03 3.21e+00 5.44e+04  -3.8 2.63e+06  -6.9 3.79e-09 4.36e-09f  1
1689  2.2893150e+03 3.21e+00 5.44e+04  -3.8 1.34e+05  -6.5 1.05e-05 1.06e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1690  2.2890596e+03 3.21e+00 5.44e+04  -3.8 1.49e+04  -7.0 4.27e-06 1.41e-05f  1
1691  2.2889735e+03 3.21e+00 5.44e+04  -3.8 8.48e+04  -5.6 4.99e-07 4.93e-07f  1
1692  2.2856770e+03 3.21e+00 5.44e+04  -3.8 2.50e+04  -5.2 6.42e-05 6.48e-05f  1
1693  2.2854516e+03 3.21e+00 5.44e+04  -3.8 8.72e+03  -5.7 4.38e-06 1.37e-05f  1
1694  2.2824170e+03 3.21e+00 5.44e+04  -3.8 5.91e+04  -5.3 2.71e-05 2.61e-05f  1
1695  2.2821931e+03 3.21e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1790  5.7318813e+02 6.53e-01 4.46e+04  -3.8 4.65e+02  -3.9 1.00e-01 7.15e-05f  1
1791  5.6806744e+02 6.51e-01 4.46e+04  -3.8 1.62e+03  -4.4 8.21e-03 3.98e-03f  1
1792  5.2920328e+02 5.43e-01 4.38e+04  -3.8 1.67e+04  -4.9 8.70e-08 3.13e-03f  1
1793  5.2919005e+02 5.43e-01 4.53e+04  -3.8 2.39e+03  -4.4 2.04e-02 8.94e-06f  1
1794  5.2918283e+02 5.43e-01 4.53e+04  -3.8 3.89e+02  -4.0 5.37e-03 2.04e-05f  1
1795  5.2740466e+02 5.41e-01 4.53e+04  -3.8 2.67e+03  -4.5 1.05e-03 1.08e-03f  1
1796  5.1120141e+02 6.99e-01 4.33e+04  -3.8 4.80e+02  -4.1 4.06e-02 4.31e-02f  1
1797  5.1119926e+02 6.95e-01 4.29e+04  -3.8 6.94e-01   0.9 7.97e-02 5.91e-03h  1
1798  5.1108889e+02 7.40e-01 4.07e+04  -3.8 7.93e-01   0.4 3.73e-02 7.37e-02f  1
1799  5.1106622e+02 7.31e-01 3.50e+04  -3.8 7.88e-01   0.8 1.50e-01 5.04e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1800  5.1106492e+02 7.13e-01

1909  8.5812775e+01 1.01e+01 9.00e+02  -3.8 1.57e+01  -2.2 3.41e-01 6.26e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1910  8.5812595e+01 1.01e+01 8.95e+02  -3.8 2.29e+01  -2.7 4.92e-03 2.77e-04h  1
1911  8.5595059e+01 1.01e+01 4.41e+02  -3.8 1.05e+03    -  1.00e-01 8.22e-03f  1
1912  8.4161988e+01 9.91e+00 1.60e+02  -3.8 7.67e+02    -  1.49e-01 5.19e-02f  1
1913  8.1345740e+01 1.04e+01 2.86e+02  -3.8 8.41e+02    -  3.06e-02 9.97e-02f  1
1914  8.1343938e+01 1.04e+01 2.86e+02  -3.8 2.25e+03    -  1.47e-05 2.61e-05h  1
1915  8.1343015e+01 1.04e+01 2.86e+02  -3.8 1.73e+03    -  2.11e-06 1.75e-05f  1
1916  8.1342926e+01 1.04e+01 2.86e+02  -3.8 3.10e+04    -  2.49e-09 9.29e-08f  2
1917  8.1342962e+01 1.04e+01 2.54e+03  -3.8 1.52e-01   2.3 4.66e-02 7.21e-05h  1
1918  8.1235449e+01 9.54e+00 1.83e+03  -3.8 4.27e+01    -  2.74e-01 8.59e-02f  1
1919  8.1180028e+01 9.50e+00 1.83e+03  -3.8 1.32e+04    -  6.13e-05 7.35e-03h  1
iter    objective    inf_pr 

2021  6.7928377e+01 6.92e-03 1.60e+01  -3.8 7.13e+00  -2.3 8.66e-02 8.32e-03h  3
2022  6.7927702e+01 3.83e-07 3.22e+00  -3.8 1.46e-01  -1.9 1.00e+00 1.00e+00H  1
2023  6.7927609e+01 1.13e-06 6.46e-01  -3.8 8.45e-02  -1.5 1.00e+00 1.00e+00h  1
2024  6.7927505e+01 1.32e-01 8.20e+01  -3.8 3.24e+00  -1.9 1.47e-01 1.21e-02h  3
2025  6.7927563e+01 8.80e-08 1.64e-02  -3.8 1.47e-02  -0.6 1.00e+00 1.00e+00h  1
2026  6.7927331e+01 5.09e-07 2.33e-01  -3.8 5.08e-02  -1.1 1.00e+00 1.00e+00h  1
2027  6.7925303e+01 2.95e-03 4.07e+00  -3.8 4.45e+02    -  1.33e-01 1.19e-01h  1
2028  6.7918614e+01 5.94e-02 1.23e+01  -3.8 6.75e+02    -  3.40e-01 2.96e-01h  1
2029  6.7919789e+01 8.10e-08 5.49e-02  -3.8 7.88e-02  -1.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2030  6.7918987e+01 9.97e-03 2.31e+00  -3.8 3.19e+02    -  2.44e-01 2.18e-01h  1
2031  6.7915341e+01 1.72e+00 2.29e+01  -3.8 9.21e+02    -  2.01e-01 3.01e-01h  1
2032  6.7915873e+01 2.24e-06

In [5]:
solve_pyomo(para_truth[1])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 3.3824864e+04 9.19e+04 8.79e+04  -4.5 6.81e+03    -  2.87e-03 2.42e-06f  1
 188r 3.3846952e+04 9.19e+04 8.78e+04  -4.5 7.85e+03  -2.8 4.71e-08 1.12e-03f  1
 189r 3.3846952e+04 9.19e+04 8.81e+04  -4.5 3.36e+04    -  1.67e-03 1.33e-08f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 3.3878029e+04 9.19e+04 8.79e+04  -4.5 3.17e+04    -  6.17e-08 7.19e-04f  1
 191r 3.3878409e+04 9.19e+04 8.87e+04  -4.5 6.89e+03    -  6.88e-03 1.21e-05f  1
 192r 3.3966570e+04 9.18e+04 8.82e+04  -4.5 6.78e+03    -  3.57e-07 2.82e-03f  1
 193r 3.3966682e+04 9.18e+04 8.82e+04  -4.5 6.83e+03    -  1.85e-04 3.61e-06f  1
 194r 3.3972655e+04 9.18e+04 8.87e+04  -4.5 6.74e+03    -  5.44e-03 1.92e-04f  1
 195r 3.4199662e+04 9.18e+04 8.75e+04  -4.5 6.74e+03    -  1.62e-03 7.32e-03f  1
 196r 3.4435249e+04 9.17e+04 8.64e+04  -4.5 6.71e+03    -  3.14e-03 7.62e-03f  1
 197r 3.4435264e+04 9.17e+04 8.65e+04  -4.5 6.63e+03    -  1.11e-03 4.82e-07f  1
 198r 3.4831455e+04 9.16e+04

 307r 4.1221696e+04 8.99e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 2.47e-09R  2
 308r 4.1219534e+04 8.99e+04 9.99e+02   5.0 5.82e+08    -  7.58e-09 1.50e-09f  1
 309r 4.0875413e+04 9.00e+04 9.99e+02   2.9 1.68e+06    -  1.02e-06 6.62e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 310r 4.1001374e+04 9.00e+04 1.40e+03   2.9 3.08e+05    -  4.05e-03 1.05e-04f  1
 311r 4.1931543e+04 8.97e+04 1.40e+03   2.9 2.91e+05    -  7.12e-04 8.31e-04f  1
 312r 4.3798047e+04 8.93e+04 1.40e+03   2.9 3.22e+05    -  1.27e-03 1.49e-03f  1
 313r 4.5046475e+04 8.89e+04 1.69e+03   2.9 3.56e+05    -  2.24e-03 9.04e-04f  1
 314r 4.7430559e+04 8.83e+04 4.16e+03   2.9 3.97e+05    -  4.36e-03 1.56e-03f  1
 315r 4.9483488e+04 8.78e+04 6.01e+03   2.9 4.73e+05    -  2.70e-03 1.12e-03f  1
 316r 5.0701369e+04 8.75e+04 7.68e+03   2.9 4.75e+05    -  1.67e-03 6.66e-04f  1
 317r 5.1618931e+04 8.72e+04 1.39e+04   2.9 3.99e+05    -  3.49e-03 6.10e-04f  1
 318r 5.2426585e+04 8.70e+04

 612r 4.0350717e+05 1.86e+03 9.99e+02   3.3 0.00e+00  -1.8 0.00e+00 2.76e-07R  2
 613r 4.0351812e+05 1.86e+03 7.41e+04   3.3 1.35e+06    -  3.73e-03 2.65e-06f  1
 614r 4.1114885e+05 5.17e+02 1.36e+05   2.6 5.15e+05    -  4.02e-02 3.58e-03f  1
 615  4.1115133e+05 5.17e+02 1.29e+04  -1.0 5.17e+02  -1.4 2.99e-03 4.37e-04h  1
 616  4.1115234e+05 5.17e+02 1.29e+04  -1.0 5.17e+02  -1.0 1.13e-02 1.87e-04h  1
 617  4.1115361e+05 5.16e+02 1.29e+04  -1.0 7.05e+02  -1.5 3.65e-03 4.15e-04h  1
 618  4.1115365e+05 5.16e+02 1.29e+04  -1.0 1.51e+03  -1.9 2.95e-03 6.54e-04h  1
 619  4.1115379e+05 5.16e+02 1.29e+04  -1.0 1.72e+03  -1.5 2.92e-03 1.20e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  4.1115322e+05 5.16e+02 1.29e+04  -1.0 2.88e+03  -2.0 4.47e-03 2.72e-04h  1
 621  4.1115135e+05 5.16e+02 1.29e+04  -1.0 4.26e+03  -2.5 7.29e-04 2.88e-04f  1
 622  4.1115071e+05 5.16e+02 1.29e+04  -1.0 3.81e+03  -2.0 1.36e-03 1.54e-04f  1
 623  4.1115062e+05 5.16e+02

 729  4.0628874e+05 2.11e+02 1.24e+06  -1.0 2.23e+04  -2.6 6.02e-02 9.01e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  4.0614134e+05 2.07e+02 1.20e+06  -1.0 2.41e+04  -3.1 5.25e-02 1.80e-02f  1
 731  4.0594435e+05 2.04e+02 1.17e+06  -1.0 2.56e+04  -3.6 2.33e-02 1.72e-02f  1
 732  4.0594089e+05 2.04e+02 1.15e+06  -1.0 2.37e+04  -3.2 2.56e-02 4.36e-04f  1
 733  4.0578930e+05 2.01e+02 1.12e+06  -1.0 2.47e+04  -3.6 2.28e-02 1.34e-02f  1
 734  4.0578581e+05 2.01e+02 1.12e+06  -1.0 2.65e+04  -4.1 4.35e-03 1.94e-04f  1
 735  4.0564207e+05 1.99e+02 1.10e+06  -1.0 2.52e+04  -3.7 1.18e-02 1.22e-02f  1
 736  4.0557988e+05 1.97e+02 1.01e+06  -1.0 2.37e+04  -3.3 1.30e-01 7.13e-03f  1
 737  4.0535849e+05 1.94e+02 1.01e+06  -1.0 2.55e+04  -3.7 3.23e-03 1.67e-02f  1
 738  4.0534990e+05 1.94e+02 1.01e+06  -1.0 2.42e+04  -3.3 1.15e-03 9.54e-04f  1
 739  4.0528876e+05 1.93e+02 1.00e+06  -1.0 2.53e+04  -3.8 7.33e-03 4.53e-03f  1
iter    objective    inf_pr 

...

5657  2.6811819e+02 3.71e-02 2.55e+04  -5.7 1.64e+01  -1.9 5.16e-03 1.68e-01f  1
5658  2.6809396e+02 3.70e-02 2.54e+04  -5.7 6.09e+00  -1.5 1.18e-01 3.35e-03h  1
5659  2.6795597e+02 3.68e-02 2.52e+04  -5.7 1.87e+01  -1.9 2.16e-03 5.59e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5660  2.6671691e+02 3.12e-02 2.14e+04  -5.7 6.86e+00  -1.5 9.71e-02 1.53e-01f  1
5661  2.6601434e+02 3.04e-02 2.06e+04  -5.7 2.13e+01  -2.0 5.27e-02 2.43e-02f  1
5662  2.6573574e+02 2.95e-02 2.00e+04  -5.7 7.67e+00  -1.6 3.15e-02 3.16e-02f  1
5663  2.6540817e+02 2.92e-02 1.97e+04  -5.7 2.69e+01  -2.0 1.14e-02 8.35e-03f  1
5664  2.6531992e+02 2.89e-02 1.95e+04  -5.7 8.60e+00  -1.6 5.35e-02 1.04e-02f  1
5665  2.6528557e+02 2.86e-02 1.94e+04  -5.7 3.23e+00  -1.2 8.02e-02 1.04e-02h  1
5666  2.6485151e+02 2.73e-02 1.85e+04  -5.7 9.69e+00  -1.7 7.02e-02 4.52e-02f  1
5667  2.6479372e+02 2.69e-02 1.82e+04  -5.7 3.63e+00  -1.2 4.20e-02 1.63e-02h  1
5668  2.6426239e+02 2.55e-02

5773  1.8733306e+02 7.67e-02 6.41e+02  -5.7 1.65e+00  -1.2 3.80e-02 8.17e-01f  1
5774  1.8721518e+02 7.29e-02 1.15e+03  -5.7 6.24e+00  -1.7 5.61e-01 6.53e-02f  1
5775  1.8685671e+02 7.41e-02 1.46e+03  -5.7 7.52e+00  -2.2 8.48e-01 2.24e-01f  1
5776  1.8584019e+02 3.21e-02 6.77e+02  -5.7 6.49e+00  -2.7 4.55e-01 6.14e-01f  1
5777  1.8508238e+02 4.02e-02 2.98e+03  -5.7 2.31e+01  -3.2 7.23e-01 1.41e-01f  1
5778  1.7121785e+02 6.76e-01 9.54e+01  -5.7 5.31e+01  -3.6 9.74e-01 1.00e+00f  1
5779  1.6442952e+02 6.62e-01 2.47e+03  -5.7 1.29e+02  -4.1 5.47e-02 2.23e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5780  1.6428599e+02 6.61e-01 2.24e+03  -5.7 4.15e+02  -4.6 6.42e-02 1.50e-03f  1
5781  1.4397982e+02 2.33e+00 9.74e+03  -5.7 4.42e+02  -5.1 4.82e-02 2.02e-01f  1
5782  1.4384206e+02 2.33e+00 9.73e+03  -5.7 7.17e+02  -2.8 2.01e-04 1.58e-04f  1
5783  1.4372231e+02 2.20e+00 1.86e+03  -5.7 1.84e+01  -3.3 7.59e-01 5.40e-02h  1
5784  1.4372011e+02 2.20e+00

5897  7.1800077e+01 1.93e-04 4.64e+01  -5.7 7.68e-02   0.2 1.00e+00 6.51e-01h  1
5898  7.1766095e+01 1.81e-04 3.87e+02  -5.7 8.13e-01  -0.3 1.00e+00 6.30e-02h  1
5899  7.1648587e+01 1.52e-04 4.65e+01  -5.7 9.08e-02   0.1 1.00e+00 6.96e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
5900  7.1597153e+01 1.53e-04 6.46e+02  -5.7 9.30e-01  -0.3 9.46e-01 8.52e-02h  1
5901  7.1467623e+01 1.87e-04 7.11e+01  -5.7 1.25e-01   0.1 1.00e+00 6.93e-01h  1
5902  7.1443718e+01 1.87e-04 7.07e+02  -5.7 6.38e+01  -0.4 1.06e-02 1.98e-03h  1
5903  7.1310772e+01 1.71e-04 4.55e+02  -5.7 4.40e-01   0.0 1.00e+00 4.54e-01h  1
5904  7.1222298e+01 4.87e-05 3.60e-01  -5.7 3.12e-02   0.5 1.00e+00 1.00e+00h  1
5905  7.1154444e+01 7.01e-05 1.81e+02  -5.7 1.34e-01  -0.0 1.00e+00 2.85e-01h  1
5906  7.1144894e+01 6.34e-05 1.11e+02  -5.7 4.35e-02   0.4 1.00e+00 1.06e-01h  1
5907  7.1123674e+01 6.20e-05 2.87e+02  -5.7 1.65e-01  -0.1 1.00e+00 8.57e-02h  1
5908  7.1042620e+01 6.91e-05

6012  6.7688408e+01 1.25e-09 7.26e-05  -5.7 2.97e-02  -2.6 1.00e+00 1.00e+00h  1
6013  6.7688306e+01 9.91e-09 6.38e-05  -5.7 7.83e-02  -3.1 1.00e+00 1.00e+00h  1
6014  6.7688003e+01 8.92e-08 6.24e-05  -5.7 2.30e-01  -3.6 1.00e+00 1.00e+00h  1
6015  6.7687610e+01 2.04e-07 2.49e+00  -5.7 6.81e-01  -4.0 1.00e+00 4.38e-01h  1
6016  6.7687070e+01 5.19e-07 9.22e+00  -5.7 1.91e+00  -4.5 1.00e+00 2.24e-01f  1
6017  6.7685798e+01 3.02e-06 2.48e+01  -5.7 4.80e+00  -5.0 1.00e+00 2.00e-01f  1
6018  6.7667250e+01 4.73e-02 2.38e-02  -5.7 1.40e+01  -5.5 1.00e+00 1.00e+00f  1
6019  6.7659955e+01 4.80e-02 2.13e+02  -5.7 3.93e+01  -6.0 1.00e+00 1.43e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
6020  6.7632761e+01 1.42e-01 2.80e+02  -5.7 9.94e+01  -6.4 4.91e-01 2.13e-01h  1
6021  6.7633330e+01 5.24e-11 2.35e-02  -5.7 9.03e-04   1.4 1.00e+00 1.00e+00h  1
6022  6.7633315e+01 6.13e-11 3.29e-03  -5.7 3.79e-04   0.9 1.00e+00 1.00e+00h  1
6023  6.7633315e+01 1.46e-11

6126  6.7347959e+01 2.30e-02 1.61e+02  -8.6 9.24e+01  -7.1 1.00e+00 4.60e-03h  1
6127  6.7322538e+01 7.94e-01 1.61e+02  -8.6 2.29e+02  -7.6 4.45e-01 2.53e-01h  1
6128  6.7323088e+01 3.90e-01 2.86e+00  -8.6 4.63e-03   0.3 1.00e+00 5.09e-01h  1
6129  6.7323531e+01 4.88e-02 3.57e-01  -8.6 2.61e-03  -0.2 1.00e+00 8.76e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
6130  6.7323596e+01 4.54e-06 5.62e-05  -8.6 2.56e-04  -0.7 1.00e+00 1.00e+00h  1
6131  6.7323595e+01 1.46e-11 1.07e-05  -8.6 1.47e-04  -1.1 1.00e+00 1.00e+00h  1
6132  6.7323595e+01 1.46e-11 9.11e-06  -8.6 3.76e-04  -1.6 1.00e+00 1.00e+00h  1
6133  6.7323595e+01 1.46e-11 8.67e-06  -8.6 1.07e-03  -2.1 1.00e+00 1.00e+00h  1
6134  6.7323593e+01 3.23e-11 8.36e-06  -8.6 3.10e-03  -2.6 1.00e+00 1.00e+00h  1
6135  6.7323593e+01 3.77e-11 1.15e-02  -8.6 9.20e-03  -3.0 1.00e+00 2.10e-01h  1
6136  6.7323581e+01 2.59e-09 8.10e-06  -8.6 2.71e-02  -3.5 1.00e+00 1.00e+00f  1
6137  6.7323546e+01 2.34e-08

In [6]:
solve_pyomo(para_truth[2])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.1068559e+04 9.73e+04 7.66e+02  -3.0 1.78e+02    -  2.18e-01 2.96e-01f  1
 188r 1.1273406e+04 9.72e+04 3.60e+02  -3.0 1.25e+02    -  3.47e-01 3.20e-01f  1
 189r 1.1488012e+04 9.72e+04 2.06e+03  -3.0 8.52e+01    -  1.77e-02 4.95e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.1601692e+04 9.72e+04 2.60e+03  -3.0 4.29e+01    -  1.10e-01 5.26e-01f  1
 191r 1.1630742e+04 9.72e+04 2.31e+03  -3.0 2.10e+01    -  1.55e-01 2.83e-01f  1
 192r 1.1635731e+04 9.72e+04 1.35e+03  -3.0 3.32e+00  -1.2 4.19e-01 5.19e-01f  1
 193r 1.1686563e+04 9.72e+04 1.17e+03  -3.0 1.38e+01    -  3.17e-01 7.42e-01f  1
 194r 1.1687701e+04 9.72e+04 4.19e+02  -3.0 8.17e-01  -0.7 6.43e-01 9.70e-01h  1
 195r 1.1688224e+04 9.72e+04 3.06e+02  -3.0 1.47e+00  -1.2 2.64e-01 1.78e-01f  1
 196r 1.1689319e+04 9.72e+04 2.49e+01  -3.0 5.40e-01  -0.8 9.19e-01 9.33e-01f  1
 197r 1.1697684e+04 9.72e+04 6.20e+01  -3.0 5.27e+00    -  1.80e-01 5.20e-01f  1
 198r 1.1699687e+04 9.72e+04

 505r 1.6539132e+05 4.52e+04 9.99e+02   4.7 0.00e+00   0.7 0.00e+00 2.78e-07R  6
 506r 1.6539162e+05 4.52e+04 1.05e+05   4.7 4.81e+07    -  9.68e-05 5.38e-09f  2
 507r 1.7126648e+05 4.30e+04 9.67e+04   2.6 8.44e+05    -  2.05e-03 2.69e-03f  1
 508r 1.9331006e+05 3.48e+04 4.27e+04   2.6 9.63e+05    -  5.43e-03 9.10e-03f  1
 509  1.9332502e+05 3.48e+04 4.59e+03  -1.0 3.48e+04   0.2 1.85e-05 3.50e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  1.9333448e+05 3.48e+04 4.59e+03  -1.0 3.48e+04  -0.3 3.92e-05 2.31e-05h  1
 511  1.9334029e+05 3.48e+04 4.57e+03  -1.0 3.48e+04   0.1 4.50e-05 1.38e-05h  1
 512r 1.9334029e+05 3.48e+04 9.99e+02   4.5 0.00e+00  -0.3 0.00e+00 2.01e-07R  2
 513r 1.9333678e+05 3.48e+04 1.91e+04   4.5 1.83e+07    -  3.19e-05 4.44e-08f  1
 514r 1.9550610e+05 3.40e+04 2.41e+04   2.4 1.08e+06    -  1.18e-03 7.67e-04f  1
 515r 2.0917699e+05 2.90e+04 1.56e+04   2.4 1.15e+06    -  1.40e-03 4.54e-03f  1
 516  2.0918324e+05 2.90e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  2.9171365e+05 4.53e+01 6.31e+05  -1.0 6.92e+02  -1.6 2.21e-03 5.41e-03h  1
 631  2.9171374e+05 4.53e+01 4.33e+05  -1.0 4.56e+02  -2.1 9.76e-02 3.48e-04h  1
 632  2.9172061e+05 4.25e+01 5.13e+05  -1.0 5.07e+02  -2.6 2.11e-02 6.21e-02h  1
 633  2.9172154e+05 4.23e+01 6.36e+05  -1.0 4.07e+02  -2.2 5.77e-01 3.72e-03h  1
 634  2.9172852e+05 3.50e+01 2.98e+05  -1.0 5.07e+02  -2.6 1.84e-02 1.73e-01h  1
 635  2.9175664e+05 2.96e+01 1.50e+05  -1.0 3.58e+02  -2.2 5.72e-02 1.52e-01h  1
 636  2.9175864e+05 2.92e+01 1.43e+05  -1.0 4.55e+02  -1.8 1.75e-03 1.55e-02h  1
 637  2.9175926e+05 2.91e+01 8.80e+04  -1.0 2.67e+02  -1.4 7.90e-02 2.28e-03h  1
 638  2.9176182e+05 2.89e+01 8.31e+04  -1.0 2.50e+02  -0.9 5.47e-03 7.93e-03h  1
 639  2.9176200e+05 2.89e+01 1.01e+05  -1.0 2.76e+02  -1.4 4.89e-02 7.65e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 640  2.9177697e+05 2.66e+01

 748  2.6859570e+05 2.08e+01 8.60e+02  -1.0 8.48e+01  -2.8 4.87e-02 1.58e-01f  1
 749  2.6853160e+05 6.10e+00 2.27e+03  -1.0 4.49e+01  -2.4 2.71e-01 7.07e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  2.6839586e+05 4.22e+00 1.93e+03  -1.0 1.31e+02  -2.9 2.90e-01 3.08e-01f  1
 751  2.6837801e+05 3.75e+00 2.16e+03  -1.0 4.94e+01  -2.5 9.47e-02 1.11e-01f  1
 752  2.6835495e+05 3.59e+00 1.60e+03  -1.0 3.31e+02  -2.9 5.85e-02 4.35e-02f  1
 753  2.6797075e+05 4.77e+00 5.07e+03  -1.0 4.50e+02  -3.4 4.39e-01 2.45e-01f  1
 754  2.6796206e+05 4.76e+00 8.27e+03  -1.0 7.87e+02  -3.9 7.75e-02 2.43e-03f  1
 755  2.6686216e+05 1.01e+01 3.44e+03  -1.0 2.39e+03  -4.4 7.48e-04 9.51e-02f  1
 756  2.6677895e+05 1.01e+01 1.76e+04  -1.0 1.40e+03  -3.9 3.16e-01 1.51e-02f  1
 757  2.6606781e+05 1.74e+01 1.51e+04  -1.0 4.76e+03  -4.4 1.17e-03 3.93e-02f  1
 758  2.6606286e+05 1.73e+01 1.80e+04  -1.0 3.53e+02  -3.1 6.67e-02 6.19e-03f  1
 759  2.6556459e+05 2.05e+01

...

1251  3.8650350e+04 5.63e+00 4.49e+04  -5.7 2.16e+01   0.6 3.85e-02 2.01e-02h  1
1252  3.8650359e+04 5.63e+00 4.31e+04  -5.7 2.18e+01   0.1 1.04e-01 5.07e-05h  1
1253  3.8654932e+04 5.49e+00 4.45e+04  -5.7 2.12e+01   0.5 7.57e-02 2.49e-02h  1
1254  3.8657312e+04 5.41e+00 4.26e+04  -5.7 2.14e+01   0.1 6.87e-02 1.39e-02h  1
1255  3.8661706e+04 5.28e+00 4.78e+04  -5.7 2.05e+01   0.5 1.89e-01 2.50e-02h  1
1256  3.8664047e+04 5.20e+00 4.52e+04  -5.7 2.08e+01   0.0 7.78e-02 1.44e-02h  1
1257  3.8669227e+04 5.04e+00 4.92e+04  -5.7 1.99e+01   0.4 1.93e-01 3.07e-02h  1
1258  3.8673818e+04 4.89e+00 4.48e+04  -5.7 2.03e+01  -0.0 1.14e-01 2.99e-02h  1
1259  3.8675277e+04 4.84e+00 4.35e+04  -5.7 2.23e+01  -0.5 2.67e-02 1.15e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1260  3.8676586e+04 4.79e+00 4.13e+04  -5.7 1.97e+01  -0.1 6.10e-02 8.93e-03h  1
1261  3.8679273e+04 4.71e+00 4.31e+04  -5.7 1.86e+01   0.3 2.08e-01 1.74e-02h  1
1262  3.8685958e+04 4.49e+00

1373  3.8639062e+04 3.41e-02 2.19e+03  -5.7 9.73e+02  -2.5 2.56e-02 2.81e-03f  1
1374  3.8638616e+04 3.41e-02 2.19e+03  -5.7 1.65e+02  -2.1 1.92e-03 8.08e-04f  1
1375  3.8636512e+04 3.37e-02 2.16e+03  -5.7 6.20e+01  -1.7 9.12e-03 1.11e-02f  1
1376  3.8635832e+04 3.34e-02 2.71e+03  -5.7 2.32e+01  -1.2 9.33e-02 9.61e-03f  1
1377  3.8627550e+04 7.07e-02 2.50e+03  -5.7 6.97e+01  -1.7 2.30e-02 3.96e-02f  1
1378  3.8625509e+04 7.11e-02 2.41e+03  -5.7 2.61e+01  -1.3 2.25e-02 2.59e-02f  1
1379  3.8618011e+04 2.11e-01 3.32e+03  -5.7 7.84e+01  -1.8 2.05e-01 3.25e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1380  3.8617403e+04 2.10e-01 3.63e+03  -5.7 2.94e+01  -1.3 7.53e-02 6.78e-03f  1
1381  3.8617402e+04 2.10e-01 3.57e+03  -5.7 6.12e+02  -1.8 1.68e-03 1.07e-06f  1
1382  3.8613387e+04 2.26e-01 3.20e+03  -5.7 3.31e+01  -1.4 7.59e-04 4.15e-02f  1
1383  3.8613344e+04 2.26e-01 5.47e+03  -5.7 1.24e+01  -1.0 3.24e-01 1.19e-03f  1
1384  3.8606812e+04 3.45e-01

1487r 3.3607040e+04 1.03e+00 1.00e+03   0.0 0.00e+00   1.8 0.00e+00 3.63e-07R  2
1488r 3.3595940e+04 1.03e+00 1.00e+03   0.0 4.53e+06    -  3.01e-04 7.30e-07f  1
1489r 3.3611996e+04 8.64e-01 9.99e+02   0.0 8.40e+04    -  8.29e-04 2.05e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1490  3.3610688e+04 8.64e-01 4.80e+03  -5.7 6.18e+03    -  1.76e-02 7.75e-05f  1
1491  3.3537447e+04 8.59e-01 5.39e+03  -5.7 5.93e+03    -  1.99e-01 4.55e-03f  1
1492  3.3441042e+04 8.52e-01 5.32e+03  -5.7 6.23e+03    -  2.84e-03 5.57e-03f  1
1493  3.2261508e+04 2.45e+00 4.31e+03  -5.7 6.21e+03    -  1.49e-02 6.85e-02f  1
1494  3.1947187e+04 2.57e+00 4.38e+03  -5.7 5.92e+03    -  3.50e-02 1.90e-02f  1
1495  3.0768653e+04 4.72e+00 3.84e+03  -5.7 5.88e+03    -  2.31e-02 7.17e-02f  1
1496  2.9102686e+04 9.16e+00 3.45e+03  -5.7 5.59e+03    -  7.27e-02 1.06e-01f  1
1497  2.9035318e+04 9.11e+00 3.44e+03  -5.7 4.40e+03    -  2.25e-02 4.84e-03f  1
1498  2.8614702e+04 8.82e+00

1654  9.2685645e+01 1.05e+01 2.05e+02  -5.7 2.19e+00  -0.2 4.18e-02 2.28e-02f  1
1655  9.0292771e+01 1.11e+01 1.20e+02  -5.7 4.28e+02    -  4.11e-02 1.08e-01f  1
1656  8.9832586e+01 1.10e+01 1.41e+02  -5.7 1.03e+03    -  3.77e-02 2.36e-02f  1
1657  8.9779213e+01 1.10e+01 9.63e+01  -5.7 1.43e+03    -  2.29e-02 2.58e-03h  1
1658  8.9773967e+01 1.10e+01 9.36e+01  -5.7 2.63e+03    -  2.81e-03 2.66e-04h  1
1659  8.9528405e+01 1.14e+01 1.06e+02  -5.7 2.94e+03    -  3.79e-03 1.19e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1660  8.9519802e+01 1.13e+01 1.11e+02  -5.7 9.44e-01  -0.7 7.30e-03 2.03e-03h  1
1661  8.9434434e+01 1.10e+01 9.10e+01  -5.7 8.44e-01  -0.3 2.49e-02 3.32e-02h  1
1662  8.9431986e+01 1.10e+01 1.33e+02  -5.7 1.11e+00  -0.7 3.47e-02 5.11e-04h  1
1663  8.9374623e+01 1.07e+01 3.47e+02  -5.7 4.85e-01  -0.3 1.43e-01 2.52e-02f  1
1664  8.8917363e+01 9.90e+00 2.60e+02  -5.7 1.08e+00  -0.8 3.22e-02 8.53e-02h  1
1665  8.8913849e+01 9.88e+00

1777  7.5946152e+01 1.30e+00 2.38e+03  -5.7 1.43e-02  -1.2 6.78e-01 1.28e-03h  1
1778  7.5945127e+01 1.29e+00 3.31e+03  -5.7 8.52e+02    -  2.94e-02 1.50e-03h  1
1779  7.5945118e+01 1.29e+00 3.31e+03  -5.7 7.46e+02    -  6.60e-05 1.48e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1780  7.5945134e+01 1.29e+00 1.83e+03  -5.7 2.29e+03    -  1.54e-02 8.37e-06f  9
1781  7.5944686e+01 1.21e+00 1.45e+03  -5.7 1.11e+01    -  2.11e-01 6.57e-02h  1
1782  7.5944687e+01 1.21e+00 2.14e+02  -5.7 1.66e-02  -1.7 1.00e+00 1.43e-03h  1
1783  7.5944271e+01 1.14e+00 1.50e+02  -5.7 1.19e+01    -  2.83e-01 5.50e-02h  1
1784  7.5944276e+01 1.13e+00 2.71e+02  -5.7 4.25e-02  -2.1 5.09e-01 6.58e-03h  1
1785  7.5944273e+01 1.13e+00 1.09e+03  -5.7 8.16e+01    -  3.58e-01 3.33e-05h  1
1786  7.5944279e+01 1.13e+00 2.00e+02  -5.7 3.50e-02  -2.6 1.00e+00 6.51e-03h  1
1787  7.5943065e+01 7.25e-01 2.53e+02  -5.7 7.67e+00    -  4.63e-02 3.56e-01h  1
1788  7.5943071e+01 7.15e-01

In [7]:
solve_pyomo(para_truth[3])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 186r 2.2182582e+04 9.46e+04 1.23e+04  -6.7 2.72e-01   2.9 9.80e-04 5.32e-07h  1
 187r 2.2182777e+04 9.46e+04 1.11e+04  -6.7 3.55e-01   2.4 3.72e-03 9.17e-02f  1
 188r 2.2182791e+04 9.46e+04 1.11e+04  -6.7 5.97e-01   1.9 1.95e-02 4.36e-03f  1
 189r 2.2186396e+04 9.46e+04 4.88e+03  -6.7 1.67e+00   1.4 6.83e-03 5.61e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 2.2188315e+04 9.46e+04 4.27e+03  -6.7 5.03e+00   1.0 2.05e-01 1.23e-01f  1
 191r 2.2188315e+04 9.46e+04 4.27e+03  -6.7 9.96e-02   3.2 4.34e-03 1.10e-06h  1
 192r 2.2188512e+04 9.46e+04 5.24e+03  -6.7 1.38e-01   2.7 1.10e-02 2.39e-01f  1
 193r 2.2188527e+04 9.46e+04 5.09e+03  -6.7 2.63e-01   2.2 3.84e-02 1.24e-02f  1
 194r 2.2189751e+04 9.46e+04 2.88e+03  -6.7 7.91e-01   1.8 2.49e-01 4.37e-01f  1
 195r 2.2189906e+04 9.46e+04 3.11e+03  -6.7 2.39e+00   1.3 2.61e-01 2.09e-02f  1
 196r 2.2190115e+04 9.46e+04 3.20e+03  -6.7 7.20e+00   0.8 1.19e-01 9.62e-03f  1
 197r 2.2191979e+04 9.46e+04

 497r 1.3064378e+05 4.79e+04 9.99e+02   4.7 0.00e+00   0.6 0.00e+00 2.84e-07R  6
 498r 1.3064379e+05 4.79e+04 6.87e+04   4.7 1.99e+07    -  1.56e-04 3.06e-08f  2
 499r 1.3869664e+05 4.44e+04 5.64e+04   2.6 8.37e+05    -  1.99e-03 4.20e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500r 1.3953152e+05 4.41e+04 7.21e+04   2.6 9.28e+05    -  3.19e-03 3.86e-04f  1
 501r 1.4345643e+05 4.23e+04 7.33e+04   2.6 9.92e+05    -  2.02e-03 1.74e-03f  1
 502  1.4346729e+05 4.23e+04 3.54e+03  -1.0 4.23e+04   0.1 2.75e-05 2.36e-05h  1
 503  1.4347717e+05 4.23e+04 3.54e+03  -1.0 4.23e+04   0.5 2.40e-05 2.03e-05h  1
 504  1.4348676e+05 4.23e+04 3.53e+03  -1.0 4.23e+04   0.0 5.21e-05 2.08e-05h  1
 505  1.4349942e+05 4.23e+04 3.52e+03  -1.0 4.23e+04   0.5 3.43e-05 2.66e-05h  1
 506  1.4350789e+05 4.23e+04 3.50e+03  -1.0 4.23e+04  -0.0 3.99e-05 1.89e-05h  1
 507r 1.4350789e+05 4.23e+04 9.99e+02   4.6 0.00e+00   0.4 0.00e+00 4.38e-07R  4
 508r 1.4350561e+05 4.23e+04

 625  2.3052496e+05 7.20e+02 2.36e+06  -1.0 2.38e+03  -3.2 2.42e-01 2.05e-03f  1
 626  2.3048052e+05 7.00e+02 2.34e+06  -1.0 2.21e+03  -2.8 7.30e-03 2.82e-02f  1
 627  2.3048024e+05 7.00e+02 2.48e+06  -1.0 1.91e+03  -1.5 9.27e-03 3.03e-04h  1
 628  2.3047884e+05 6.99e+02 2.50e+06  -1.0 2.41e+03  -1.9 6.38e-03 1.24e-03f  1
 629  2.3047229e+05 6.95e+02 2.48e+06  -1.0 2.07e+03  -2.4 4.15e-03 5.86e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  2.3047097e+05 6.95e+02 2.48e+06  -1.0 1.66e+04  -2.9 1.34e-05 1.38e-04f  1
 631  2.3047026e+05 6.95e+02 2.47e+06  -1.0 2.46e+03  -2.5 1.78e-02 4.44e-04f  1
 632  2.3045483e+05 6.87e+02 2.42e+06  -1.0 2.02e+03  -2.0 3.52e-03 1.16e-02f  1
 633  2.3037781e+05 6.59e+02 2.32e+06  -1.0 2.40e+03  -2.5 4.18e-02 4.01e-02f  1
 634  2.3037182e+05 6.56e+02 2.41e+06  -1.0 2.01e+03  -2.1 5.88e-02 4.62e-03f  1
 635  2.3035064e+05 6.48e+02 2.40e+06  -1.0 2.21e+03  -2.6 2.45e-03 1.24e-02f  1
 636  2.3035033e+05 6.48e+02

 744  2.1444084e+05 2.99e+01 1.26e+05  -1.0 1.85e+02  -2.2 5.74e-03 1.98e-02f  1
 745  2.1441204e+05 2.81e+01 1.80e+05  -1.0 1.97e+02  -2.7 2.55e-01 6.24e-02f  1
 746  2.1434679e+05 2.67e+01 1.53e+05  -1.0 2.91e+02  -3.1 8.58e-03 4.87e-02f  1
 747  2.1429149e+05 2.42e+01 1.57e+05  -1.0 1.84e+02  -2.7 1.69e-01 9.50e-02f  1
 748  2.1427755e+05 2.42e+01 1.56e+05  -1.0 6.47e+03  -3.2 2.96e-04 2.29e-03f  1
 749  2.1422963e+05 2.24e+01 1.31e+05  -1.0 1.92e+02  -2.8 1.16e-02 7.61e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  2.1419979e+05 2.21e+01 1.34e+05  -1.0 5.82e+02  -3.2 2.86e-02 1.40e-02f  1
 751  2.1414686e+05 2.04e+01 1.49e+05  -1.0 1.77e+02  -2.8 1.81e-01 7.47e-02f  1
 752  2.1414701e+05 2.04e+01 1.06e+05  -1.0 6.83e+01  -0.6 1.24e-02 1.73e-03h  1
 753  2.1414795e+05 2.00e+01 1.42e+05  -1.0 8.18e+01  -1.1 8.52e-04 1.78e-02h  1
 754  2.1414777e+05 1.99e+01 1.31e+05  -1.0 1.19e+02  -1.5 1.56e-02 4.53e-03h  1
 755  2.1414658e+05 1.97e+01

...

1962  1.0739070e+02 1.88e+00 4.26e+04  -5.7 1.86e+02    -  7.85e-02 1.35e-01h  1
1963  1.0789577e+02 1.82e+00 4.28e+04  -5.7 5.23e+01    -  5.17e-02 3.26e-02h  1
1964  1.0953530e+02 1.66e+00 3.76e+04  -5.7 5.45e+01    -  7.04e-02 8.84e-02h  1
1965  1.0950974e+02 1.62e+00 3.57e+04  -5.7 2.11e+02    -  3.13e-04 2.36e-02h  1
1966  1.0934816e+02 1.44e+00 2.84e+04  -5.7 2.09e+02    -  6.80e-02 1.27e-01h  1
1967  1.0931460e+02 1.42e+00 3.03e+04  -5.7 2.11e+02    -  6.62e-02 1.42e-02h  1
1968  1.0881931e+02 1.34e+00 2.01e+04  -5.7 2.21e+02    -  2.51e-02 1.59e-01h  1
1969  1.0847325e+02 1.29e+00 1.46e+04  -5.7 2.00e+02    -  3.22e-02 1.12e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1970  1.0788145e+02 1.34e+00 1.17e+04  -5.7 1.89e+02    -  1.39e-01 1.85e-01f  1
1971  1.0787777e+02 1.34e+00 1.17e+04  -5.7 1.99e+02    -  5.86e-03 7.01e-04h  1
1972  1.0765605e+02 1.30e+00 1.45e+04  -5.7 2.00e+02    -  1.77e-01 4.19e-02f  1
1973  1.0751940e+02 1.28e+00

2108  8.0243696e+01 3.15e-01 2.76e+03  -8.6 6.00e-03   0.1 9.94e-01 1.84e-01h  1
2109  8.0243405e+01 2.83e-01 2.57e+03  -8.6 7.21e-02  -0.4 3.45e-01 6.57e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2110  8.0242826e+01 1.99e-01 2.13e+03  -8.6 7.98e-03   0.0 1.00e+00 1.77e-01h  1
2111  8.0242780e+01 1.98e-01 2.11e+03  -8.6 1.25e+00  -0.4 3.43e-02 4.55e-03h  1
2112  8.0240379e+01 2.36e-04 1.88e+02  -8.6 5.76e-03  -0.0 1.00e+00 9.11e-01h  1
2113  8.0240141e+01 6.56e-06 5.12e+00  -8.6 2.25e-03   0.4 1.00e+00 9.72e-01h  1
2114  8.0240116e+01 5.59e-06 1.59e+02  -8.6 1.18e-02  -0.1 1.00e+00 1.48e-01h  1
2115  8.0240088e+01 3.48e-07 2.89e+00  -8.6 2.43e-03   0.4 1.00e+00 9.38e-01f  1
2116  8.0239982e+01 9.39e-08 6.93e+01  -8.6 1.05e-02  -0.1 1.00e+00 7.30e-01f  1
2117  8.0239937e+01 1.43e-08 1.29e+01  -8.6 3.41e-03   0.3 1.00e+00 8.47e-01f  1
2118  8.0239705e+01 8.48e-09 1.47e+02  -8.6 2.31e-02  -0.2 1.00e+00 7.43e-01h  1
2119  8.0239484e+01 6.49e-10

2224  7.9716120e+01 7.30e+00 5.92e+03  -8.6 4.74e+02  -7.9 1.96e-01 5.01e-02h  1
2225  7.9715697e+01 7.28e+00 8.13e+03  -8.6 3.81e+02  -7.5 2.82e-01 1.88e-03h  1
2226  7.9708538e+01 7.24e+00 7.14e+03  -8.6 1.06e+03    -  7.20e-02 7.64e-03h  1
2227r 7.9708538e+01 7.24e+00 1.00e+03   0.9 0.00e+00    -  0.00e+00 2.22e-10R  7
2228r 7.9708538e+01 7.24e+00 1.24e+04   0.9 6.40e+10    -  1.04e-11 8.20e-17f  1
2229r 7.9799561e+01 6.97e+00 1.24e+04   0.9 1.17e+06    -  4.55e-06 3.36e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2230r 8.0809987e+01 1.04e+01 1.23e+04   0.9 3.79e+05    -  1.43e-03 5.92e-04f  1
2231r 8.2601695e+01 4.16e-01 1.23e+04   0.9 5.37e+04    -  2.59e-03 2.42e-03f  1
2232r 8.6816410e+01 1.05e+01 1.22e+04   0.9 3.71e+04    -  7.31e-03 6.64e-03f  1
2233r 1.1059785e+02 2.37e+02 1.19e+04   0.9 2.08e+04    -  8.80e-03 4.79e-02f  1
2234r 1.1125280e+02 2.30e+02 8.78e+03   0.9 7.49e+00   2.0 3.18e-01 3.20e-02f  1
2235r 1.3745755e+02 2.03e+02

2352  7.9718971e+01 2.52e-09 9.51e-01  -8.6 3.66e-02  -2.8 1.00e+00 5.73e-01h  1
2353  7.9718944e+01 5.17e-09 4.40e+00  -8.6 1.08e-01  -3.3 1.00e+00 2.32e-01f  1
2354  7.9718938e+01 5.28e-09 1.51e+01  -8.6 3.23e-01  -3.8 1.00e+00 2.01e-02f  1
2355  7.9718864e+01 2.53e-07 3.41e+01  -8.6 9.68e-01  -4.2 1.00e+00 9.52e-02f  1
2356  7.9718261e+01 5.32e-05 8.40e+01  -8.6 2.90e+00  -4.7 1.00e+00 2.60e-01f  1
2357  7.9718161e+01 5.39e-05 3.11e+02  -8.6 8.59e+00  -5.2 1.00e+00 1.53e-02f  1
2358  7.9717651e+01 1.02e-04 8.37e+02  -8.6 1.55e+01  -5.7 8.92e-01 3.10e-02h  1
2359  7.9717651e+01 1.02e-04 4.85e+00  -8.6 9.17e-04   0.2 1.00e+00 1.96e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2360  7.9717651e+01 8.92e-05 3.97e-01  -8.6 1.22e-04  -0.3 1.00e+00 1.25e-01f  4
2361  7.9717651e+01 1.46e-11 5.90e-05  -8.6 3.61e-04  -0.8 1.00e+00 1.00e+00h  1
2362  7.9717650e+01 1.46e-11 2.52e-02  -8.6 1.02e-03  -1.3 1.00e+00 6.21e-01h  1
2363  7.9717650e+01 2.91e-11

2467  7.9699456e+01 5.13e-04 2.10e+03  -8.6 1.04e+02  -7.1 5.15e-03 6.69e-02f  1
2468  7.9698094e+01 6.30e-04 6.92e+03  -8.6 3.25e+02  -7.6 2.30e-01 6.82e-03f  1
2469  7.9694084e+01 1.90e-03 7.11e+03  -8.6 1.25e+03  -8.1 7.89e-03 5.22e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2470  7.9617676e+01 2.22e+00 5.09e+02  -8.6 7.68e+03  -8.5 2.83e-03 1.61e-02f  1
2471r 7.9617676e+01 2.22e+00 1.00e+03   0.3 0.00e+00  -9.0 0.00e+00 1.23e-09R  6
2472r 7.9626111e+01 9.00e+00 1.00e+03   0.3 2.06e+08    -  4.25e-09 1.57e-06f  1
2473r 7.9724883e+01 8.87e+00 1.00e+03   0.3 3.10e+05    -  9.21e-05 6.83e-05f  1
2474r 8.0384260e+01 1.99e+01 1.01e+03   0.3 1.48e+05    -  5.22e-04 8.40e-04f  1
2475r 8.1136383e+01 5.50e+01 1.00e+03   0.3 6.42e+04    -  3.44e-03 1.61e-03f  1
2476r 8.4280011e+01 1.14e+02 1.00e+03   0.3 3.94e+04    -  5.92e-03 9.41e-03f  1
2477r 9.2800908e+01 3.14e+02 1.01e+03   0.3 1.82e+04    -  6.00e-03 3.50e-02f  1
2478r 9.3034657e+01 3.05e+02

In [8]:
solve_pyomo(para_truth[4])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 186r 1.2238270e+04 9.70e+04 3.48e+03  -6.7 1.58e+02  -0.6 7.50e-03 1.11e-02f  1
 187r 1.2238319e+04 9.70e+04 3.47e+03  -6.7 4.66e+02  -1.1 2.10e-04 3.17e-05f  1
 188r 1.2272590e+04 9.70e+04 3.70e+03  -6.7 1.38e+03  -1.5 4.02e-03 7.40e-03f  1
 189r 1.2272637e+04 9.70e+04 3.69e+03  -6.7 5.25e+02  -1.1 2.35e-04 2.77e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.2298148e+04 9.70e+04 3.92e+03  -6.7 1.55e+03  -1.6 1.57e-03 4.98e-03f  1
 191r 1.2302178e+04 9.70e+04 3.94e+03  -6.7 6.36e+02  -1.2 2.41e-05 2.47e-03f  1
 192r 1.2345691e+04 9.70e+04 4.45e+03  -6.7 2.39e+02  -0.7 1.52e-02 7.11e-02f  1
 193r 1.2369790e+04 9.70e+04 4.27e+03  -6.7 7.15e+02  -1.2 2.27e-02 1.32e-02f  1
 194r 1.2369790e+04 9.70e+04 4.15e+03  -6.7 2.54e+02  -0.8 1.92e-03 2.63e-07f  1
 195r 1.2458106e+04 9.69e+04 6.29e+03  -6.7 7.48e+02  -1.3 2.32e-03 3.65e-02f  1
 196r 1.2462342e+04 9.69e+04 6.21e+03  -6.7 2.85e+02  -0.8 5.48e-03 4.69e-03f  1
 197r 1.2494698e+04 9.69e+04

 505  1.6799220e+05 1.06e+02 2.26e+05  -1.0 4.12e+02  -0.6 3.80e-02 9.72e-03h  1
 506  1.6802352e+05 1.04e+02 2.24e+05  -1.0 4.53e+02  -1.1 2.79e-02 1.75e-02h  1
 507  1.6802928e+05 1.04e+02 2.80e+05  -1.0 4.49e+02  -0.7 7.69e-02 3.11e-03h  1
 508  1.6804309e+05 1.03e+02 2.79e+05  -1.0 4.81e+02  -1.1 2.17e-02 8.89e-03h  1
 509  1.6805334e+05 1.02e+02 3.30e+05  -1.0 4.63e+02  -0.7 7.32e-02 6.21e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  1.6808540e+05 9.99e+01 3.23e+05  -1.0 4.67e+02  -1.2 4.09e-02 2.25e-02h  1
 511  1.6810569e+05 9.83e+01 3.14e+05  -1.0 4.57e+02  -1.7 2.87e-02 1.56e-02h  1
 512  1.6812040e+05 9.74e+01 4.27e+05  -1.0 4.35e+02  -0.3 5.96e-02 8.68e-03h  1
 513  1.6813481e+05 9.64e+01 4.26e+05  -1.0 4.42e+02  -0.8 1.69e-02 1.05e-02h  1
 514  1.6815960e+05 9.45e+01 4.12e+05  -1.0 4.40e+02  -1.3 5.54e-02 1.95e-02h  1
 515  1.6817408e+05 9.35e+01 4.26e+05  -1.0 4.28e+02  -0.9 5.19e-02 1.10e-02h  1
 516  1.6819245e+05 9.21e+01

 626  1.6872962e+05 1.13e+01 1.81e+04  -1.0 2.51e+02  -1.8 3.07e-04 4.47e-03h  1
 627  1.6872957e+05 1.13e+01 5.34e+04  -1.0 1.58e+02  -1.4 7.39e-03 5.20e-04h  1
 628  1.6872937e+05 1.12e+01 5.43e+04  -1.0 1.45e+02  -1.0 4.38e-03 4.30e-03h  1
 629  1.6872905e+05 1.12e+01 1.15e+05  -1.0 3.79e+02  -1.5 6.34e-03 1.27e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  1.6872849e+05 1.12e+01 4.57e+05  -1.0 3.49e+02  -1.0 1.39e-02 2.62e-03h  1
 631  1.6872303e+05 1.10e+01 4.16e+05  -1.0 4.13e+02  -1.5 1.43e-02 1.72e-02f  1
 632  1.6872001e+05 1.08e+01 3.09e+05  -1.0 3.96e+02  -1.1 8.06e-03 1.15e-02h  1
 633  1.6871857e+05 1.08e+01 3.36e+05  -1.0 3.95e+02  -1.6 7.56e-03 4.35e-03f  1
 634  1.6871696e+05 1.07e+01 7.99e+06  -1.0 3.94e+02  -1.1 3.11e-01 6.04e-03h  1
 635  1.6215926e+05 2.13e+01 8.09e+06  -1.0 6.03e+04    -  1.36e-03 3.85e-02f  1
 636  1.6064658e+05 2.20e+01 7.75e+06  -1.0 6.26e+04    -  3.88e-02 8.57e-03f  1
 637  1.6042511e+05 2.20e+01

 749  2.0809121e+04 4.21e+01 1.08e+04  -1.0 5.36e+04  -4.8 4.18e-02 3.01e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  2.0416911e+04 4.20e+01 1.06e+04  -1.0 1.15e+05  -5.3 2.62e-04 1.29e-03f  1
 751  1.9722357e+04 4.06e+01 1.02e+04  -1.0 6.05e+03  -4.0 8.56e-02 3.52e-02f  1
 752  1.8312635e+04 3.92e+01 9.44e+03  -1.0 1.87e+04  -4.5 1.22e-03 2.41e-02f  1
 753  1.3928669e+04 4.82e+01 9.44e+03  -1.0 6.94e+03  -4.0 6.32e-02 1.96e-01f  1
 754  1.3895698e+04 4.82e+01 1.09e+04  -1.0 2.81e+04  -4.5 1.00e-02 2.82e-04f  1
 755  1.3829974e+04 4.82e+01 1.06e+04  -1.0 1.91e+04  -5.0 1.05e-04 1.10e-03f  1
 756  1.3484078e+04 4.88e+01 1.07e+04  -1.0 3.44e+03  -3.7 1.54e-02 2.68e-02f  1
 757  1.3195781e+04 4.61e+01 9.06e+03  -1.0 1.31e+03  -3.2 1.31e-01 6.82e-02f  1
 758  1.3172178e+04 4.60e+01 1.29e+04  -1.0 3.44e+03  -3.7 1.85e-01 2.19e-03f  1
 759  1.2691072e+04 4.43e+01 1.05e+04  -1.0 6.55e+03  -4.2 2.14e-03 3.06e-02f  1
iter    objective    inf_pr 

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 870  1.7226938e+03 2.41e-01 2.60e+02  -3.8 5.79e-01   0.0 1.00e+00 2.55e-01f  1
 871  1.7220303e+03 2.19e-01 7.60e+02  -3.8 1.75e+00  -0.4 3.95e-01 9.43e-02f  1
 872  1.7210106e+03 1.37e-01 1.10e+03  -3.8 6.59e-01  -0.0 1.00e+00 3.74e-01f  1
 873  1.7200700e+03 1.20e-01 1.16e+03  -3.8 1.98e+00  -0.5 3.26e-01 1.28e-01f  1
 874  1.7192927e+03 8.40e-02 1.83e+03  -3.8 7.53e-01  -0.1 1.00e+00 3.01e-01f  1
 875  1.7122092e+03 5.85e-03 4.63e+00  -3.8 2.26e+00  -0.5 1.00e+00 1.00e+00f  1
 876  1.7112567e+03 3.96e-03 9.50e+01  -3.8 8.46e-01  -0.1 1.00e+00 3.54e-01f  1
 877  1.7104513e+03 8.45e-04 4.14e+01  -3.8 3.21e-01   0.3 1.00e+00 8.18e-01f  1
 878  1.7091533e+03 1.06e-03 5.36e+01  -3.8 9.57e-01  -0.2 1.00e+00 4.37e-01f  1
 879  1.7086083e+03 3.03e-03 6.44e+00  -3.8 5.54e+00  -0.6 3.23e-03 5.63e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 880  1.7085784e+03 3.01e-03

 989  1.1651614e+03 3.27e-02 2.16e+03  -3.8 7.15e+02  -3.5 1.57e-02 7.17e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 990  1.1648433e+03 3.27e-02 2.16e+03  -3.8 1.18e+03  -3.9 2.88e-05 1.83e-04f  1
 991  1.1585069e+03 3.25e-02 2.22e+03  -3.8 8.34e+02  -3.5 1.31e-02 5.66e-03f  1
 992  1.1581975e+03 3.25e-02 2.22e+03  -3.8 1.24e+03  -4.0 2.67e-05 1.69e-04f  1
 993  1.1523345e+03 3.24e-02 2.29e+03  -3.8 1.00e+03  -3.6 1.13e-02 4.52e-03f  1
 994  1.1520443e+03 3.24e-02 2.29e+03  -3.8 1.30e+03  -4.0 2.52e-05 1.52e-04f  1
 995  1.1463974e+03 3.23e-02 2.38e+03  -3.8 1.25e+03  -3.6 9.60e-03 3.65e-03f  1
 996  1.1461396e+03 3.23e-02 2.38e+03  -3.8 1.36e+03  -4.1 2.36e-05 1.29e-04f  1
 997  1.1404742e+03 3.22e-02 2.49e+03  -3.8 1.63e+03  -3.7 8.05e-03 2.93e-03f  1
 998  1.1402550e+03 3.22e-02 2.49e+03  -3.8 1.42e+03  -4.1 2.21e-05 1.05e-04f  1
 999  1.1343529e+03 3.21e-02 2.63e+03  -3.8 2.30e+03  -3.7 6.46e-03 2.28e-03f  1
iter    objective    inf_pr 

1117  3.9764499e+02 9.95e+00 3.29e+03  -3.8 2.85e+04  -7.5 8.68e-05 2.05e-05f  1
1118  3.9763366e+02 9.95e+00 3.14e+03  -3.8 1.11e+04  -7.1 1.95e-02 5.43e-05f  1
1119  3.9740213e+02 9.95e+00 3.14e+03  -3.8 4.13e+04    -  9.07e-06 1.44e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1120  3.9701679e+02 9.95e+00 3.13e+03  -3.8 9.22e+03    -  2.40e-03 3.09e-04f  1
1121  3.9677824e+02 9.94e+00 3.13e+03  -3.8 4.74e+03    -  1.94e-05 9.16e-04f  1
1122  3.9631028e+02 9.94e+00 3.13e+03  -3.8 6.14e+03    -  9.23e-04 2.75e-04f  1
1123  3.9626483e+02 9.94e+00 3.13e+03  -3.8 2.70e+03    -  1.66e-05 2.73e-04f  1
1124  3.9567607e+02 9.93e+00 3.14e+03  -3.8 5.07e+03    -  6.80e-04 2.76e-04f  1
1125  3.9565205e+02 9.93e+00 3.14e+03  -3.8 1.69e+03    -  1.96e-05 2.32e-04h  1
1126  3.9499112e+02 9.93e+00 3.15e+03  -3.8 4.29e+03    -  7.94e-04 3.27e-04f  1
1127  3.9497235e+02 9.92e+00 3.15e+03  -3.8 1.05e+03    -  2.64e-05 3.09e-04h  1
1128  3.9423678e+02 9.92e+00

1247  1.0919812e+02 4.42e-08 1.53e-01  -3.8 1.23e-03   2.1 1.00e+00 1.00e+00f  1
1248  1.0919832e+02 2.64e-07 6.31e-01  -3.8 5.55e-03   1.6 1.00e+00 1.00e+00h  1
1249  1.0919764e+02 5.26e-08 2.45e-01  -3.8 2.19e-03   2.0 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1250  1.0919608e+02 2.95e-07 1.78e+02  -3.8 9.48e-03   1.6 1.00e+00 5.43e-01h  1
1251  1.0919567e+02 2.14e-08 1.95e-01  -3.8 1.97e-03   2.0 1.00e+00 1.00e+00f  1
1252  1.0919545e+02 3.38e-07 1.22e-01  -3.8 3.73e-03   1.5 1.00e+00 1.00e+00h  1
1253  1.0919496e+02 7.18e-08 1.54e-01  -3.8 1.75e-03   1.9 1.00e+00 1.00e+00h  1
1254  1.0919295e+02 1.74e-06 2.56e-01  -3.8 8.76e-03   1.5 1.00e+00 1.00e+00h  1
1255  1.0919055e+02 3.31e-07 3.03e-01  -3.8 3.86e-03   1.9 1.00e+00 1.00e+00h  1
1256  1.0918403e+02 3.88e-04 2.55e+02  -3.8 2.23e-02   1.4 1.00e+00 5.20e-01h  1
1257  1.0912172e+02 4.58e-04 7.15e+02  -3.8 1.28e+05    -  1.82e-04 7.80e-06f  1
1258  1.0677630e+02 1.42e-01

1383  7.4182274e+01 2.79e+00 7.25e+01  -5.7 4.92e-02  -1.2 1.00e+00 6.36e-01h  1
1384  7.4182538e+01 2.49e-02 2.77e-02  -5.7 1.79e-02  -1.7 1.00e+00 1.00e+00h  1
1385  7.4182535e+01 5.27e-09 1.22e-02  -5.7 5.89e-03  -1.3 1.00e+00 1.00e+00h  1
1386  7.4182170e+01 6.23e-01 5.29e+02  -5.7 1.37e-01  -1.8 5.91e-01 1.00e+00H  1
1387  7.4181777e+01 9.03e-01 1.03e+02  -5.7 8.41e-02  -1.3 1.00e+00 8.66e-01h  1
1388  7.4180844e+01 3.86e+00 1.27e+03  -5.7 1.60e+02    -  1.00e+00 5.81e-01h  1
1389  7.4180902e+01 2.43e+00 3.71e+02  -5.7 3.42e+01    -  9.12e-01 4.81e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1390  7.4180850e+01 1.73e-03 1.93e-02  -5.7 1.56e+01    -  1.00e+00 1.00e+00h  1
1391  7.4180847e+01 6.11e-10 1.35e-04  -5.7 6.19e-01    -  1.00e+00 1.00e+00h  1
1392  7.4180847e+01 1.46e-11 1.37e-09  -5.7 1.06e-03    -  1.00e+00 1.00e+00h  1
1393  7.4180395e+01 6.20e-03 8.22e-02  -8.6 2.11e+00    -  9.92e-01 9.92e-01f  1
1394  7.4180392e+01 2.62e-10

In [9]:
solve_pyomo(para_truth[5])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.0680175e+04 9.74e+04 4.84e+00  -3.0 3.64e+00   0.1 1.00e+00 1.00e+00f  1
 188r 1.0688746e+04 9.74e+04 1.10e+02  -3.0 1.06e+01  -0.4 4.86e-01 1.93e-01f  1
 189r 1.0689679e+04 9.74e+04 1.33e+02  -3.0 4.02e+00   0.1 1.00e+00 5.53e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.0738891e+04 9.74e+04 4.63e+00  -3.0 1.17e+01  -0.4 1.00e+00 9.89e-01f  1
 191r 1.0747921e+04 9.74e+04 1.15e+03  -3.0 3.18e+01  -0.9 1.00e+00 6.53e-02f  1
 192r 1.0757623e+04 9.74e+04 2.03e+03  -3.0 8.21e+01  -1.4 4.01e-01 2.69e-02f  1
 193r 1.0790471e+04 9.74e+04 1.69e+03  -3.0 1.73e+02  -1.8 2.78e-04 4.17e-02f  1
 194r 1.0960342e+04 9.73e+04 7.82e+02  -3.0 2.75e+02  -2.3 6.15e-02 1.20e-01f  1
 195r 1.0970720e+04 9.73e+04 1.03e+03  -3.0 3.27e+02  -2.8 2.15e-02 5.97e-03f  1
 196r 1.0972685e+04 9.73e+04 8.66e+02  -3.0 1.86e+00   0.3 1.00e+00 2.49e-01f  1
 197r 1.0996191e+04 9.73e+04 1.57e+01  -3.0 5.48e+00  -0.1 9.65e-01 1.00e+00f  1
 198r 1.1001210e+04 9.73e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 2.9544649e+04 9.13e+04 2.67e+03  -6.7 5.16e+02  -1.1 1.38e-03 1.32e-01f  1
 371r 2.9544649e+04 9.13e+04 6.11e+03  -6.7 1.53e+03  -1.6 4.42e-02 1.39e-08f  1
 372r 2.9672284e+04 9.13e+04 4.99e+03  -6.7 5.77e+02  -1.2 7.93e-05 6.41e-02f  1
 373r 2.9672284e+04 9.13e+04 5.05e+03  -6.7 2.17e+02  -0.7 5.04e-02 3.88e-08f  1
 374r 2.9673499e+04 9.13e+04 5.03e+03  -6.7 6.47e+02  -1.2 9.94e-09 5.49e-04f  1
 375r 2.9712421e+04 9.13e+04 4.42e+03  -6.7 2.47e+02  -0.8 2.67e-01 6.75e-02f  1
 376r 2.9712422e+04 9.13e+04 4.42e+03  -6.7 7.39e+02  -1.3 6.82e-08 3.76e-07f  1
 377r 2.9754379e+04 9.12e+04 4.13e+03  -6.7 2.78e+02  -0.8 5.63e-02 6.51e-02f  1
 378r 2.9754379e+04 9.12e+04 4.56e+03  -6.7 8.27e+02  -1.3 2.44e-02 5.37e-09f  1
 379r 2.9761784e+04 9.12e+04 6.84e+03  -6.7 3.07e+02  -0.9 9.76e-02 6.95e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 2.9761828e+04 9.12e+04

 636  2.3484428e+05 6.19e+01 1.49e+04  -1.0 1.21e+03  -0.2 2.20e-03 9.65e-04h  1
 637  2.3484513e+05 6.19e+01 1.61e+04  -1.0 1.15e+03  -0.7 1.31e-03 4.04e-04h  1
 638  2.3484533e+05 6.19e+01 4.80e+04  -1.0 1.19e+03  -0.2 8.65e-03 1.00e-04h  1
 639  2.3484595e+05 6.18e+01 5.91e+04  -1.0 1.24e+03  -0.7 8.12e-03 1.18e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 640  2.3484673e+05 6.17e+01 1.11e+05  -1.0 1.20e+03  -0.3 1.17e-02 1.07e-03h  1
 641  2.3484662e+05 6.17e+01 1.54e+05  -1.0 1.20e+03  -0.8 2.39e-02 1.22e-03h  1
 642  2.3484658e+05 6.16e+01 2.60e+05  -1.0 1.17e+03  -0.3 1.99e-02 7.50e-04h  1
 643  2.3484457e+05 6.14e+01 2.56e+05  -1.0 1.15e+03  -0.8 1.09e-03 2.63e-03f  1
 644  2.3484443e+05 6.14e+01 3.36e+05  -1.0 1.13e+03  -0.4 1.42e-02 2.71e-04h  1
 645  2.3484138e+05 6.12e+01 3.30e+05  -1.0 1.12e+03  -0.9 3.03e-04 3.17e-03f  1
 646  2.3484148e+05 6.12e+01 6.14e+05  -1.0 1.10e+03   0.5 8.11e-03 1.62e-04h  1
 647  2.3484123e+05 6.12e+01

 755  2.3330766e+05 7.09e+00 2.61e+06  -1.0 1.85e+02   0.8 5.97e-03 6.38e-03h  1
 756  2.3330738e+05 7.07e+00 2.13e+07  -1.0 3.22e+02   0.4 1.56e-01 2.26e-03h  1
 757  2.3329664e+05 6.37e+00 1.79e+07  -1.0 2.40e+02  -0.1 2.59e-03 9.88e-02f  1
 758  2.3329655e+05 6.37e+00 9.87e+06  -1.0 2.41e+02  -0.6 3.73e-01 8.79e-04h  1
 759  2.3329643e+05 6.36e+00 1.01e+07  -1.0 2.96e+02   1.6 1.60e-03 1.11e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  2.3329611e+05 6.34e+00 3.71e+08  -1.0 1.86e+02   1.2 5.15e-01 3.70e-03h  1
 761  2.3329564e+05 6.31e+00 3.27e+08  -1.0 2.12e+02   0.7 3.80e-01 5.01e-03h  1
 762  2.3329075e+05 5.97e+00 6.87e+07  -1.0 2.15e+02   0.2 9.91e-01 5.26e-02f  1
 763  2.3329199e+05 5.94e+00 6.84e+07  -1.0 6.37e+02  -0.3 3.52e-03 5.20e-03h  1
 764  2.3327925e+05 4.67e+00 7.16e+07  -1.0 1.84e+02  -0.7 7.35e-04 2.14e-01f  1
 765  2.3324786e+05 3.86e+00 5.93e+07  -1.0 4.11e+02  -1.2 1.90e-01 2.47e-01f  1
 766  2.3324758e+05 3.86e+00

...

1221  5.7448978e+04 9.36e+01 3.96e+03  -3.8 3.77e+04  -4.8 7.99e-02 5.38e-02f  1
1222  5.7448310e+04 9.36e+01 3.98e+03  -3.8 1.22e+04  -4.3 4.37e-02 1.96e-05f  1
1223  5.6190124e+04 9.27e+01 3.96e+03  -3.8 4.56e+04  -4.8 3.27e-06 1.02e-02f  1
1224  5.6028509e+04 9.27e+01 3.93e+03  -3.8 2.91e+06  -5.3 7.30e-05 2.13e-05f  1
1225  5.4325666e+04 9.17e+01 3.73e+03  -3.8 5.88e+04  -4.9 2.30e-02 1.08e-02f  1
1226  5.1758144e+04 8.66e+01 3.48e+03  -3.8 1.65e+04  -4.4 6.66e-02 5.77e-02f  1
1227  5.1757504e+04 8.66e+01 3.38e+03  -3.8 7.08e+04  -4.9 1.39e-02 3.37e-06f  1
1228  4.7757144e+04 8.02e+01 3.13e+03  -3.8 1.86e+04  -4.5 8.92e-06 7.94e-02f  1
1229  4.7756800e+04 8.02e+01 3.73e+03  -3.8 8.90e+02  -3.2 2.69e-01 1.54e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1230  4.7206004e+04 7.33e+01 3.08e+03  -3.8 2.35e+03  -3.6 4.14e-04 8.66e-02f  1
1231  4.5941099e+04 6.98e+01 4.13e+03  -3.8 1.40e+04  -4.1 1.16e-03 4.71e-02f  1
1232  4.5922511e+04 6.97e+01

1343  2.6604373e+03 2.90e+01 2.23e+04  -3.8 2.19e+04  -3.4 4.67e-06 6.31e-04f  1
1344  2.6604265e+03 2.90e+01 2.20e+04  -3.8 3.39e+03  -3.9 6.80e-04 7.31e-07f  1
1345  2.6464295e+03 2.89e+01 2.36e+04  -3.8 3.34e+03  -4.4 1.26e-02 1.68e-03f  1
1346  2.5563462e+03 2.88e+01 2.19e+04  -3.8 4.65e+03  -4.9 2.02e-06 7.77e-03f  1
1347  2.5426333e+03 2.88e+01 2.38e+04  -3.8 5.71e+03  -5.3 6.84e-03 8.17e-04f  1
1348  2.5426261e+03 2.88e+01 2.38e+04  -3.8 2.06e+03  -5.8 1.85e-06 3.33e-06h  1
1349  2.5365662e+03 2.87e+01 2.38e+04  -3.8 1.74e+04  -4.5 8.72e-05 1.00e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1350  2.5364999e+03 2.87e+01 2.56e+04  -3.8 1.10e+03  -5.0 1.54e-02 5.51e-05f  1
1351  2.5356362e+03 2.87e+01 2.69e+04  -3.8 1.18e+03  -5.4 1.21e-02 6.58e-04f  1
1352  2.5323780e+03 2.87e+01 2.68e+04  -3.8 2.43e+03  -5.9 1.52e-06 7.29e-04f  1
1353  2.5315568e+03 2.87e+01 2.68e+04  -3.8 2.60e+03  -3.7 4.77e-05 1.06e-04f  1
1354  2.5280268e+03 2.87e+01

1463  5.8824240e+02 7.52e+00 1.40e+04  -3.8 2.37e+03  -7.4 4.04e-02 2.92e-02f  1
1464  5.6322795e+02 7.46e+00 1.51e+04  -3.8 1.28e+03  -7.0 1.52e-01 2.54e-02f  1
1465  5.0590312e+02 7.70e+00 1.33e+04  -3.8 1.27e+03  -6.5 2.86e-05 6.03e-02f  1
1466  4.7415392e+02 7.67e+00 1.26e+04  -3.8 2.33e+03  -7.0 1.98e-02 3.70e-02f  1
1467  4.5659422e+02 7.58e+00 1.22e+04  -3.8 1.22e+03  -6.6 1.06e-02 2.18e-02f  1
1468  4.0856668e+02 7.91e+00 1.14e+04  -3.8 3.76e+03  -7.1 5.37e-02 6.01e-02f  1
1469  4.0855867e+02 7.91e+00 1.26e+04  -3.8 8.26e+02  -6.6 9.21e-02 1.56e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  4.0855111e+02 7.91e+00 1.27e+04  -3.8 4.13e+03  -7.1 8.27e-03 1.11e-05f  1
1471  4.0794612e+02 7.85e+00 1.38e+04  -3.8 3.23e+03  -6.7 4.22e-02 7.75e-03f  1
1472  4.0790019e+02 7.85e+00 1.41e+04  -3.8 8.76e+03  -7.2 2.05e-02 3.07e-04f  1
1473  4.0426856e+02 7.53e+00 1.51e+04  -3.8 3.46e+03  -6.8 9.97e-02 4.79e-02f  1
1474  4.0348465e+02 7.51e+00

1596  8.2320158e+01 6.62e+00 3.53e+02  -3.8 1.49e+03    -  1.72e-04 1.03e-04h  1
1597  8.2320016e+01 6.62e+00 3.53e+02  -3.8 2.22e+03    -  9.54e-06 4.77e-06f  2
1598  8.2320023e+01 6.62e+00 3.78e+03  -3.8 4.98e-01   2.0 6.96e-02 2.00e-05h  1
1599  8.2308368e+01 6.57e+00 3.37e+03  -3.8 1.07e+03    -  9.01e-02 6.53e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1600  8.2268119e+01 5.92e+00 3.32e+03  -3.8 8.11e+01    -  2.08e-02 9.91e-02h  1
1601  8.1988552e+01 5.87e+00 2.91e+03  -3.8 6.83e+03    -  7.07e-02 2.83e-02h  1
1602  8.1999070e+01 4.30e+00 2.05e+03  -3.8 1.95e+02    -  2.94e-01 2.69e-01h  1
1603  7.8859742e+01 5.44e+00 2.00e+03  -3.8 6.21e+02    -  5.49e-05 2.34e-01f  1
1604  7.8869127e+01 4.99e+00 3.32e+03  -3.8 4.08e-02   1.5 1.00e+00 8.28e-02h  1
1605  7.8095887e+01 5.09e+00 2.67e+03  -3.8 6.65e+02    -  1.92e-01 5.69e-02f  1
1606  7.3925291e+01 2.29e+01 2.43e+03  -3.8 9.51e+02    -  4.90e-02 3.37e-01f  1
1607  7.3924566e+01 2.29e+01

1733  6.9117913e+01 4.98e-08 4.60e-05  -8.6 2.49e-01    -  1.00e+00 1.00e+00h  1
1734  6.9117913e+01 1.46e-11 6.47e-08  -8.6 3.05e-04    -  1.00e+00 1.00e+00h  1
1735  6.9117913e+01 1.46e-11 2.60e-13  -8.6 2.21e-08    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 1735

                                   (scaled)                 (unscaled)
Objective...............:   6.9117913363515996e+01    6.9117913363515996e+01
Dual infeasibility......:   2.6029806294344121e-13    2.6029806294344121e-13
Constraint violation....:   1.4551915228366852e-11    1.4551915228366852e-11
Variable bound violation:   9.9945554055125285e-09    9.9945554055125285e-09
Complementarity.........:   2.5059286508726415e-09    2.5059286508726415e-09
Overall NLP error.......:   2.5059286508726415e-09    2.5059286508726415e-09


Number of objective function evaluations             = 1889
Number of objective gradient evaluations             = 1194
Number of equality constraint evaluations            = 1889
Number of

In [10]:
solve_pyomo(para_truth[6])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.4563829e+04 9.61e+04 6.41e+02  -6.7 1.01e+02  -0.4 1.00e+00 1.44e-06f  1
 188r 1.4908573e+04 9.60e+04 1.35e+03  -6.7 3.03e+02  -0.8 1.00e+00 3.83e-01f  1
 189r 1.4990487e+04 9.60e+04 6.44e+03  -6.7 9.07e+02  -1.3 7.15e-01 3.04e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.5704776e+04 9.57e+04 8.12e+02  -6.7 3.41e+02  -0.9 1.00e+00 7.05e-01f  1
 191r 1.6064540e+04 9.56e+04 5.40e+01  -6.7 1.28e+02  -0.5 1.00e+00 9.52e-01f  1
 192r 1.6488148e+04 9.55e+04 2.29e+03  -6.7 3.83e+02  -0.9 1.00e+00 3.73e-01f  1
 193r 1.6488149e+04 9.55e+04 1.35e+03  -6.7 1.44e+02  -0.5 1.00e+00 3.64e-07f  1
 194r 1.6522190e+04 9.55e+04 1.91e+03  -6.7 4.30e+02  -1.0 2.26e-01 2.68e-02f  1
 195r 1.6522194e+04 9.55e+04 7.56e+03  -6.7 1.28e+03  -1.5 4.33e-01 1.17e-06f  1
 196r 1.7955674e+04 9.50e+04 3.90e+02  -6.7 4.84e+02  -1.0 8.56e-01 1.00e+00f  1
 197r 1.8008169e+04 9.50e+04 1.92e+02  -6.7 1.81e+02  -0.6 5.16e-07 9.87e-02f  1
 198r 1.8198496e+04 9.49e+04

 497  2.0287845e+05 1.26e+02 8.98e+06  -1.0 4.40e+03  -1.4 1.48e-02 2.67e-03h  1
 498  2.0288570e+05 1.25e+02 9.78e+06  -1.0 2.11e+03  -1.0 8.09e-02 3.39e-03h  1
 499  2.0289339e+05 1.25e+02 9.79e+06  -1.0 1.34e+04  -1.5 1.22e-04 5.08e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500  2.0291893e+05 1.23e+02 1.08e+07  -1.0 1.94e+03  -1.1 1.36e-01 1.17e-02h  1
 501  2.0292737e+05 1.23e+02 1.12e+07  -1.0 1.84e+03  -0.6 1.52e-02 3.94e-03h  1
 502  2.0298136e+05 1.20e+02 1.09e+07  -1.0 1.86e+03  -1.1 1.53e-02 2.57e-02h  1
 503  2.0298197e+05 1.20e+02 1.30e+07  -1.0 1.79e+03  -0.7 6.32e-02 2.96e-04h  1
 504  2.0301557e+05 1.18e+02 1.28e+07  -1.0 1.78e+03  -1.2 3.04e-02 1.71e-02h  1
 505  2.0301666e+05 1.17e+02 1.43e+07  -1.0 1.74e+03  -0.7 5.48e-02 5.68e-04h  1
 506  2.0303022e+05 1.17e+02 1.41e+07  -1.0 1.75e+03  -1.2 2.80e-02 7.62e-03h  1
 507  2.0303642e+05 1.16e+02 1.43e+07  -1.0 1.70e+03  -0.8 1.43e-02 3.55e-03h  1
 508  2.0305574e+05 1.15e+02

 619  2.0373691e+05 2.43e+01 2.22e+06  -1.0 9.53e+02  -1.8 4.10e-04 5.37e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  2.0364785e+05 3.53e+01 2.26e+06  -1.0 7.21e+03  -2.3 5.96e-03 1.44e-02f  1
 621  2.0364729e+05 3.53e+01 1.68e+06  -1.0 8.97e+02  -1.9 7.95e-02 1.72e-03h  1
 622  2.0364159e+05 3.52e+01 1.65e+06  -1.0 1.80e+03  -2.4 5.55e-03 3.05e-03f  1
 623  2.0363440e+05 3.48e+01 1.72e+06  -1.0 8.32e+02  -2.0 5.92e-05 1.10e-02f  1
 624  2.0363046e+05 3.41e+01 1.55e+06  -1.0 8.76e+02  -1.5 5.28e-02 1.89e-02f  1
 625  2.0363035e+05 3.41e+01 1.28e+06  -1.0 8.28e+02  -2.0 3.70e-02 1.93e-04h  1
 626  2.0362731e+05 3.41e+01 1.29e+06  -1.0 1.30e+03  -1.6 7.41e-04 8.87e-04f  1
 627  2.0360632e+05 3.40e+01 1.22e+06  -1.0 1.93e+03  -2.1 8.25e-03 3.17e-03f  1
 628  2.0358737e+05 3.38e+01 1.28e+06  -1.0 1.22e+03  -1.6 6.80e-05 6.22e-03f  1
 629  2.0358693e+05 3.38e+01 1.27e+06  -1.0 8.59e+02  -2.1 1.53e-03 9.77e-04h  1
iter    objective    inf_pr 

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 740  1.9928393e+05 4.99e+01 1.68e+06  -1.0 1.93e+03  -3.6 4.41e-02 1.71e-03f  1
 741  1.9916410e+05 4.94e+01 1.56e+06  -1.0 3.92e+03  -4.1 7.04e-02 9.04e-03f  1
 742  1.9753366e+05 4.83e+01 1.44e+06  -1.0 1.33e+04  -4.5 7.88e-02 3.78e-02f  1
 743  1.9358641e+05 1.02e+02 1.42e+06  -1.0 7.87e+04  -5.0 1.09e-02 1.75e-02f  1
 744  1.9260706e+05 1.03e+02 1.42e+06  -1.0 1.74e+04  -4.6 1.37e-03 1.74e-02f  1
 745  1.9140777e+05 1.11e+02 1.42e+06  -1.0 1.49e+05  -5.1 1.12e-03 2.83e-03f  1
 746  1.9090206e+05 1.11e+02 1.08e+06  -1.0 2.13e+04  -4.6 2.37e-01 7.59e-03f  1
 747  1.8867425e+05 1.47e+02 1.07e+06  -1.0 2.40e+05  -5.1 7.27e-03 3.52e-03f  1
 748  1.8779972e+05 1.49e+02 9.92e+05  -1.0 2.41e+04  -4.7 7.49e-02 1.35e-02f  1
 749  1.8564104e+05 1.50e+02 9.90e+05  -1.0 2.45e+05  -5.2 1.30e-03 3.33e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  1.8520619e+05 1.49e+02

...

1211  2.9826552e+03 5.70e+00 3.99e+04  -3.8 1.05e+03  -3.4 5.84e-02 3.79e-05f  1
1212  2.7997042e+03 5.72e+00 3.76e+04  -3.8 4.22e+03  -3.9 6.61e-06 1.90e-02f  1
1213  2.7966232e+03 5.71e+00 4.25e+04  -3.8 1.15e+03  -3.5 1.78e-01 9.25e-04f  1
1214  2.6965607e+03 5.71e+00 4.13e+04  -3.8 4.53e+03  -4.0 2.05e-05 9.78e-03f  1
1215  2.6771279e+03 5.68e+00 4.19e+04  -3.8 1.33e+03  -3.5 3.39e-02 5.37e-03f  1
1216  2.6673175e+03 5.68e+00 4.20e+04  -3.8 5.80e+03  -4.0 1.60e-03 8.26e-04f  1
1217  2.6451188e+03 5.65e+00 5.03e+04  -3.8 1.42e+03  -3.6 1.97e-01 7.37e-03f  1
1218  2.6448427e+03 5.65e+00 5.74e+04  -3.8 9.61e+03  -4.1 4.63e-02 1.55e-05f  1
1219  2.4806866e+03 5.90e+00 5.69e+04  -3.8 1.59e+03  -3.6 1.21e-01 4.86e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1220  2.4806223e+03 5.90e+00 5.87e+04  -3.8 6.11e+03  -4.1 3.41e-02 5.17e-06f  1
1221  2.2671338e+03 7.21e+00 5.75e+04  -3.8 1.66e+03  -3.7 1.89e-01 6.20e-02f  1
1222  2.2668935e+03 7.21e+00

1333  5.0675276e+02 8.38e+00 2.00e+03  -3.8 7.55e+00  -1.7 6.20e-02 1.32e-01f  1
1334  5.0528108e+02 7.22e+00 2.77e+03  -3.8 3.94e+00  -1.3 8.43e-02 1.38e-01f  1
1335  5.0485159e+02 7.02e+00 2.05e+03  -3.8 7.66e+00  -1.8 6.03e-02 2.80e-02f  1
1336  5.0192823e+02 4.93e+00 3.59e+03  -3.8 3.88e+00  -1.4 1.81e-01 2.99e-01f  1
1337  5.0136670e+02 4.71e+00 3.92e+03  -3.8 7.49e+00  -1.8 1.73e-02 4.34e-02f  1
1338  5.0131933e+02 4.69e+00 8.10e+02  -3.8 3.81e+00  -1.4 2.51e-01 4.91e-03h  1
1339  5.0014982e+02 4.29e+00 7.44e+02  -3.8 8.34e+00  -1.9 8.17e-02 8.15e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1340  4.9796413e+02 3.02e+00 1.26e+03  -3.8 3.67e+00  -1.5 1.86e-01 2.96e-01f  1
1341  4.9764709e+02 2.93e+00 8.68e+02  -3.8 8.25e+00  -1.9 5.57e-02 2.76e-02f  1
1342  4.9755028e+02 2.93e+00 4.94e+02  -3.8 2.98e+01  -0.6 1.64e-05 1.20e-03f  1
1343  4.9695369e+02 2.93e+00 2.90e+02  -3.8 1.79e+03    -  1.00e-02 6.79e-04f  1
1344  4.9040344e+02 2.92e+00

1467  1.6467280e+02 6.92e+00 9.52e+03  -3.8 2.16e+03    -  2.36e-04 6.19e-04f  1
1468  1.6466379e+02 6.92e+00 9.51e+03  -3.8 5.85e+02    -  3.20e-03 3.29e-05f  1
1469  1.3733956e+02 1.47e+01 8.36e+03  -3.8 7.59e+02    -  2.31e-02 2.39e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1470  1.3738912e+02 1.45e+01 2.57e+05  -3.8 5.36e-01   1.2 9.93e-01 1.36e-02h  1
1471  1.3730266e+02 1.44e+01 2.44e+05  -3.8 5.46e+02    -  4.98e-02 8.11e-03h  1
1472  1.3695857e+02 1.44e+01 2.44e+05  -3.8 3.66e+03    -  7.54e-06 5.51e-04f  1
1473  1.3678093e+02 1.36e+01 5.41e+04  -3.8 8.23e+01    -  7.74e-01 5.34e-02h  1
1474  1.3421747e+02 5.91e+00 5.59e+04  -3.8 7.92e+01    -  5.21e-04 5.65e-01f  1
1475  1.3428099e+02 5.49e+00 8.24e+04  -3.8 1.39e-01   1.6 2.98e-01 7.25e-02h  1
1476  1.3435300e+02 4.99e+00 4.27e+05  -3.8 1.37e-01   2.1 1.00e+00 9.05e-02h  1
1477  1.3440096e+02 4.68e+00 5.58e+05  -3.8 1.45e-01   2.5 2.34e-01 6.15e-02h  1
1478  1.3444766e+02 4.38e+00

1595  9.1582987e+01 1.46e-11 3.88e-04  -5.7 1.81e-03  -0.7 1.00e+00 1.00e+00f  1
1596  9.1582890e+01 2.91e-11 3.68e-04  -5.7 5.17e-03  -1.1 1.00e+00 1.00e+00f  1
1597  9.1582616e+01 8.73e-11 3.48e-04  -5.7 1.47e-02  -1.6 1.00e+00 1.00e+00f  1
1598  9.1581806e+01 5.45e-10 3.44e-04  -5.7 4.35e-02  -2.1 1.00e+00 1.00e+00f  1
1599  9.1580629e+01 1.82e-09 5.06e+00  -5.7 1.30e-01  -2.6 1.00e+00 4.87e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1600  9.1578634e+01 9.37e-09 1.98e+01  -5.7 3.91e-01  -3.1 1.00e+00 2.87e-01f  1
1601  9.1577333e+01 1.18e-08 6.86e+01  -5.7 1.01e+00  -3.5 1.00e+00 7.02e-02f  1
1602  9.1573268e+01 1.19e-07 7.65e+01  -5.7 2.82e+00  -4.0 1.76e-01 7.98e-02f  1
1603  9.1573267e+01 9.64e-08 2.18e+01  -5.7 6.95e-03   0.9 7.61e-01 1.87e-01f  2
1604  9.1573266e+01 1.46e-11 5.78e-04  -5.7 2.03e-04   0.5 1.00e+00 1.00e+00h  1
1605  9.1573258e+01 1.46e-11 5.53e-04  -5.7 5.83e-04  -0.0 1.00e+00 1.00e+00h  1
1606  9.1573239e+01 1.46e-11

1722  8.3915434e+01 4.90e-09 7.79e+01  -5.7 1.71e-02  -0.7 1.00e+00 3.11e-01h  2
1723  8.3915407e+01 4.68e-11 4.93e-02  -5.7 4.19e-03  -0.3 1.00e+00 1.00e+00h  1
1724  8.3915370e+01 6.95e-04 1.67e-01  -5.7 1.29e-02  -0.8 1.00e+00 1.00e+00h  1
1725  8.3915335e+01 3.97e-10 3.43e-02  -5.7 5.63e-03  -0.3 1.00e+00 1.00e+00h  1
1726  8.3915224e+01 4.72e-03 4.78e+01  -5.7 3.05e-02  -0.8 1.00e+00 7.05e-01h  1
1727  8.3915105e+01 3.45e-03 4.67e+01  -5.7 2.00e+00    -  7.66e-02 2.06e-01f  1
1728  8.3915085e+01 3.04e-03 9.45e+01  -5.7 2.17e-02  -1.3 1.00e+00 9.66e-02f  1
1729  8.3911168e+01 2.55e-02 1.04e+02  -5.7 1.43e+01    -  2.20e-02 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1730  8.3891707e+01 1.07e+00 2.81e+02  -5.7 1.75e+03    -  3.38e-02 4.12e-02h  1
1731  8.3832762e+01 2.26e+01 6.70e+02  -5.7 5.84e+02    -  3.47e-01 4.12e-01h  1
1732  8.3862074e+01 9.04e+00 6.63e+02  -5.7 1.38e+02    -  2.65e-01 1.00e+00h  1
1733  8.3835982e+01 1.49e+01

In [11]:
solve_pyomo(para_truth[7])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.0696906e+04 9.74e+04 1.75e+03  -3.0 2.10e+02    -  3.49e-02 4.30e-02f  1
 188r 1.0706198e+04 9.74e+04 2.57e+02  -3.0 2.01e+02    -  1.33e-01 9.15e-03f  1
 189r 1.1068717e+04 9.73e+04 3.66e+03  -3.0 1.99e+02    -  2.08e-02 3.62e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.1075062e+04 9.73e+04 4.62e+01  -3.0 2.49e+01  -1.3 7.66e-01 5.51e-02f  1
 191r 1.1088792e+04 9.73e+04 1.46e+02  -3.0 1.26e+02    -  6.14e-03 2.17e-02f  1
 192r 1.1376849e+04 9.72e+04 1.80e+03  -3.0 1.23e+02    -  2.01e-01 4.68e-01f  1
 193r 1.1377040e+04 9.72e+04 1.15e+03  -3.0 5.58e+00  -0.9 3.18e-01 7.12e-03f  1
 194r 1.1466441e+04 9.72e+04 2.07e+03  -3.0 6.55e+01    -  4.26e-04 2.72e-01f  1
 195r 1.1468228e+04 9.72e+04 1.68e+03  -3.0 2.75e-01   0.4 1.97e-01 1.00e+00h  1
 196r 1.1470991e+04 9.72e+04 2.98e+02  -3.0 6.28e-01  -0.1 8.25e-01 1.00e+00f  1
 197r 1.1471605e+04 9.72e+04 3.19e+01  -3.0 1.79e+00  -0.5 7.25e-01 7.57e-02f  1
 198r 1.1474680e+04 9.72e+04

 498r 1.0573680e+05 3.81e+04 9.99e+02   4.6 0.00e+00   1.0 0.00e+00 4.40e-07R  3
 499r 1.0573042e+05 3.81e+04 6.80e+03   4.6 1.67e+07    -  2.20e-05 3.11e-07f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 500r 1.0916996e+05 3.57e+04 8.57e+03   2.5 2.16e+06    -  1.72e-03 1.11e-03f  1
 501r 1.1988695e+05 2.92e+04 4.28e+03   2.5 2.43e+06    -  1.36e-03 2.68e-03f  1
 502  1.1989235e+05 2.92e+04 4.27e+03  -1.0 2.92e+04   0.6 2.37e-05 1.49e-05h  1
 503  1.1989995e+05 2.92e+04 4.31e+03  -1.0 2.92e+04   0.1 5.95e-05 2.17e-05h  1
 504  1.1991879e+05 2.92e+04 4.35e+03  -1.0 2.92e+04  -0.4 1.01e-04 5.92e-05h  1
 505  1.1993739e+05 2.92e+04 4.57e+03  -1.0 2.92e+04   0.0 1.44e-04 5.73e-05h  1
 506  1.1994155e+05 2.92e+04 4.81e+03  -1.0 2.92e+04  -0.4 1.19e-04 1.54e-05h  1
 507r 1.1994155e+05 2.92e+04 9.99e+02   4.5 0.00e+00  -0.0 0.00e+00 1.45e-07R  2
 508r 1.1994098e+05 2.92e+04 1.19e+03   4.5 2.05e+07    -  3.36e-06 7.31e-08f  1
 509r 1.2095235e+05 2.85e+04

 629  1.5369232e+05 5.20e+01 8.93e+05  -1.0 9.20e+02  -1.8 3.62e-02 1.44e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  1.5369816e+05 5.15e+01 8.84e+05  -1.0 8.55e+02  -1.4 5.11e-02 8.50e-03h  1
 631  1.5370527e+05 5.05e+01 8.76e+05  -1.0 9.47e+02  -1.8 1.06e-02 2.04e-02h  1
 632  1.5370766e+05 5.03e+01 8.69e+05  -1.0 9.23e+02  -1.4 1.93e-02 3.73e-03h  1
 633  1.5371644e+05 4.97e+01 9.19e+05  -1.0 8.80e+02  -1.0 5.54e-02 1.26e-02h  1
 634  1.5372418e+05 4.90e+01 9.01e+05  -1.0 9.99e+02  -1.5 3.69e-02 1.31e-02h  1
 635  1.5372833e+05 4.87e+01 9.10e+05  -1.0 9.75e+02  -1.0 1.82e-02 6.53e-03h  1
 636  1.5373300e+05 4.83e+01 8.81e+05  -1.0 1.05e+03  -1.5 6.93e-02 8.96e-03h  1
 637  1.5373888e+05 4.71e+01 8.49e+05  -1.0 2.01e+03  -2.0 3.57e-02 2.37e-02h  1
 638  1.5373911e+05 4.71e+01 8.25e+05  -1.0 1.03e+03  -1.6 1.07e-01 4.75e-04h  1
 639  1.5374305e+05 4.62e+01 8.02e+05  -1.0 1.07e+03  -2.0 2.73e-02 1.89e-02h  1
iter    objective    inf_pr 

 751  1.4133574e+05 2.53e+01 1.34e+05  -1.0 1.28e+03  -3.1 2.41e-03 3.58e-02f  1
 752  1.4131099e+05 2.53e+01 1.33e+05  -1.0 6.37e+03  -3.6 4.41e-03 1.37e-03f  1
 753  1.4117484e+05 2.45e+01 1.23e+05  -1.0 1.38e+03  -3.1 8.19e-02 3.30e-02f  1
 754  1.4075151e+05 1.70e+01 1.07e+05  -1.0 4.88e+02  -2.7 1.25e-01 3.05e-01f  1
 755  1.3999106e+05 1.47e+01 1.04e+05  -1.0 1.87e+03  -3.2 1.66e-03 1.32e-01f  1
 756  1.3957231e+05 1.22e+01 8.02e+04  -1.0 8.33e+02  -2.8 2.53e-01 1.74e-01f  1
 757  1.3956479e+05 1.22e+01 7.93e+04  -1.0 2.14e+03  -3.2 1.43e-02 1.18e-03f  1
 758  1.3945270e+05 1.21e+01 7.87e+04  -1.0 8.78e+03  -3.7 1.07e-02 4.30e-03f  1
 759  1.3940806e+05 1.20e+01 5.91e+04  -1.0 1.70e+03  -3.3 2.13e-01 8.61e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  1.3900939e+05 1.17e+01 4.38e+04  -1.0 3.95e+03  -3.8 1.83e-01 3.18e-02f  1
 761  1.3881894e+05 1.16e+01 4.35e+04  -1.0 1.07e+04  -4.3 6.30e-03 5.42e-03f  1
 762  1.3526267e+05 1.29e+01

...

1236  1.6870834e+03 2.61e+00 2.08e+04  -3.8 3.08e+03  -5.1 6.60e-06 1.93e-05f  1
1237  1.6843481e+03 2.61e+00 2.08e+04  -3.8 1.19e+04  -4.7 1.45e-04 1.30e-04f  1
1238  1.6842446e+03 2.61e+00 2.08e+04  -3.8 3.11e+03  -5.2 6.69e-06 1.99e-05f  1
1239  1.6817096e+03 2.61e+00 2.08e+04  -3.8 2.86e+04  -4.7 6.14e-05 5.22e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1240  1.6815944e+03 2.61e+00 2.08e+04  -3.8 3.13e+03  -5.2 7.01e-06 2.21e-05f  1
1241  1.6756002e+03 2.61e+00 2.07e+04  -3.8 1.46e+03  -3.9 3.13e-03 2.48e-03f  1
1242  1.6754512e+03 2.61e+00 2.07e+04  -3.8 2.12e+03  -4.4 1.22e-05 4.23e-05f  1
1243  1.6704140e+03 2.60e+00 2.07e+04  -3.8 1.71e+03  -3.9 2.52e-03 1.82e-03f  1
1244  1.6702351e+03 2.60e+00 2.07e+04  -3.8 2.19e+03  -4.4 1.26e-05 4.92e-05f  1
1245  1.6657507e+03 2.60e+00 2.07e+04  -3.8 2.07e+03  -4.0 2.11e-03 1.38e-03f  1
1246  1.6655373e+03 2.60e+00 2.07e+04  -3.8 2.25e+03  -4.5 1.30e-05 5.71e-05f  1
1247  1.6614476e+03 2.59e+00

1366  1.8290240e+02 6.24e+00 2.08e+03  -3.8 1.30e+01  -2.2 5.41e-02 5.21e-03h  1
1367  1.8276396e+02 6.21e+00 2.15e+03  -3.8 5.92e+01  -2.7 2.02e-02 3.94e-03f  1
1368  1.8232713e+02 6.15e+00 2.33e+03  -3.8 8.26e+01  -3.2 3.04e-02 8.00e-03f  1
1369  1.8190523e+02 5.83e+00 2.99e+03  -3.8 7.62e+00  -1.9 1.89e-01 5.19e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  1.8157396e+02 5.72e+00 3.01e+03  -3.8 2.21e+01  -2.3 3.18e-02 1.75e-02f  1
1371  1.8130467e+02 5.53e+00 3.17e+03  -3.8 6.72e+00  -1.9 9.06e-02 3.27e-02h  1
1372  1.8127496e+02 5.49e+00 3.13e+03  -3.8 3.16e+00  -1.5 2.74e-03 6.71e-03h  1
1373  1.8088700e+02 5.31e+00 2.92e+03  -3.8 2.10e+01  -2.0 2.11e-03 3.01e-02f  1
1374  1.8087496e+02 5.30e+00 3.27e+03  -3.8 2.30e+00  -1.5 9.58e-02 2.80e-03h  1
1375  1.8081254e+02 5.16e+00 3.13e+03  -3.8 8.90e-01  -1.1 7.69e-03 2.60e-02h  1
1376  1.8016591e+02 4.42e+00 2.28e+03  -3.8 2.82e+00  -1.6 2.77e-02 1.42e-01h  1
1377  1.8005470e+02 4.41e+00

1489  9.5846205e+01 2.91e-11 1.28e-03  -5.7 4.77e-03  -0.6 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1490  9.5845170e+01 2.12e-10 1.27e-03  -5.7 1.42e-02  -1.0 1.00e+00 1.00e+00f  1
1491  9.5843701e+01 3.06e-10 5.09e-01  -5.7 4.26e-02  -1.5 1.00e+00 4.74e-01f  1
1492  9.5834761e+01 2.94e-08 1.26e-03  -5.7 1.27e-01  -2.0 1.00e+00 1.00e+00f  1
1493  9.5807903e+01 2.21e-05 1.23e-03  -5.7 3.71e-01  -2.5 1.00e+00 1.00e+00f  1
1494  9.5727529e+01 2.56e-04 1.22e-03  -5.7 1.11e+00  -3.0 1.00e+00 1.00e+00f  1
1495  9.5655539e+01 4.07e-04 1.01e+01  -5.7 3.32e+00  -3.4 1.77e-01 3.00e-01f  1
1496  9.5652752e+01 4.05e-04 1.01e+01  -5.7 9.88e+00  -3.9 1.65e-03 3.91e-03f  1
1497  9.5652646e+01 4.05e-04 1.01e+01  -5.7 3.04e+01  -4.4 6.45e-06 4.82e-05f  1
1498  9.5652596e+01 4.05e-04 1.01e+01  -5.7 9.02e+01  -4.9 1.22e-06 7.68e-06f  1
1499  9.5652570e+01 4.05e-04 1.01e+01  -5.7 2.12e+02  -5.3 2.56e-06 1.72e-06f  1
iter    objective    inf_pr 

1627  7.0305562e+01 2.69e+00 2.22e+02  -5.7 1.38e+02  -6.3 1.00e+00 7.14e-01f  1
1628  6.9628765e+01 8.18e+00 3.73e+02  -5.7 6.12e+02    -  2.30e-02 2.43e-01f  1
1629  6.9593295e+01 8.09e+00 1.82e+02  -5.7 6.79e+02    -  2.10e-01 1.40e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1630  6.9593396e+01 8.08e+00 1.64e+02  -5.7 1.99e-02   1.6 1.33e-02 1.05e-03h  1
1631  6.9593421e+01 8.07e+00 1.42e+03  -5.7 2.11e-02   1.1 1.00e+00 2.63e-04h  9
1632  6.9586111e+01 8.05e+00 1.40e+03  -5.7 6.37e+02    -  2.06e-02 3.13e-03h  1
1633  6.9204624e+01 9.50e+00 1.26e+03  -5.7 6.96e+02    -  6.82e-02 1.50e-01h  1
1634  6.9204465e+01 9.50e+00 1.27e+03  -5.7 9.82e+02    -  4.07e-06 5.15e-05h  1
1635  6.9204449e+01 9.50e+00 1.27e+03  -5.7 2.01e+03    -  9.68e-08 2.39e-06f  1
1636  6.9204442e+01 9.50e+00 1.27e+03  -5.7 8.02e+02    -  3.53e-06 2.73e-06f  1
1637  6.9204458e+01 9.50e+00 1.00e+03  -5.7 3.63e+02    -  2.70e-01 1.20e-05f  9
1638  6.9204311e+01 9.50e+00

1764  6.7970158e+01 3.05e-01 1.96e+03  -5.7 2.17e+00  -2.9 3.27e-01 8.17e-03h  1
1765  6.7970179e+01 3.05e-01 1.95e+03  -5.7 2.89e+00  -3.4 6.56e-03 1.11e-03h  3
1766  6.7970158e+01 3.04e-01 1.11e+03  -5.7 1.00e+00  -3.9 4.09e-01 3.58e-03h  1
1767  6.7970152e+01 2.78e-01 3.10e+02  -5.7 3.05e-01  -1.7 1.00e+00 8.67e-02h  1
1768  6.7969104e+01 2.47e-05 3.48e+02  -5.7 3.59e+01    -  4.51e-02 1.00e+00h  1
1769  6.7969109e+01 3.01e-10 1.15e+01  -5.7 7.51e-03  -2.1 9.66e-01 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1770  6.7969089e+01 6.19e-10 1.21e+02  -5.7 8.13e-02  -2.6 1.00e+00 2.10e-01h  2
1771  6.7969054e+01 4.65e-10 6.67e-02  -5.7 2.05e-02  -3.1 1.00e+00 1.00e+00h  1
1772  6.7967490e+01 4.23e-01 2.31e+01  -5.7 2.51e+05    -  1.33e-04 1.89e-04f  1
1773  6.7963964e+01 3.94e+00 9.46e+01  -5.7 8.43e+02    -  2.50e-01 1.33e-01h  1
1774  6.7964644e+01 2.12e+00 9.18e+01  -5.7 4.31e+01    -  1.89e-01 5.19e-01h  1
1775  6.7959885e+01 1.95e+01

In [12]:
solve_pyomo(para_truth[8])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 7.8124148e+03 9.80e+04 8.61e+03  -3.0 1.22e+01  -0.1 2.27e-01 1.09e-03f  1
 188r 7.8947818e+03 9.80e+04 8.56e+03  -3.0 7.25e+02    -  3.15e-02 2.38e-02f  1
 189r 7.8948728e+03 9.80e+04 6.75e+03  -3.0 4.53e+00   0.3 2.15e-01 5.00e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 7.8974285e+03 9.80e+04 7.24e+03  -3.0 7.08e+02    -  2.31e-02 7.57e-04f  1
 191r 7.9819300e+03 9.79e+04 6.52e+03  -3.0 7.07e+02    -  1.20e-06 2.51e-02f  1
 192r 7.9837806e+03 9.79e+04 5.35e+03  -3.0 1.67e+00   0.7 1.78e-01 2.54e-01f  1
 193r 7.9914500e+03 9.79e+04 3.00e+03  -3.0 4.95e+00   0.3 4.41e-01 3.90e-01f  1
 194r 8.0083287e+03 9.79e+04 8.89e+02  -3.0 1.45e+01  -0.2 7.68e-01 2.99e-01f  1
 195r 8.0134546e+03 9.79e+04 8.59e+02  -3.0 4.09e+01  -0.7 2.40e-02 3.14e-02f  1
 196r 8.0135535e+03 9.79e+04 8.56e+02  -3.0 1.61e+01  -0.3 6.33e-03 1.58e-03f  1
 197r 8.0140329e+03 9.79e+04 8.77e+02  -3.0 4.56e+01  -0.7 4.90e-02 2.67e-03f  1
 198r 8.0234186e+03 9.79e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 370r 2.8277584e+04 9.15e+04 5.59e+03  -6.7 1.84e+03  -1.7 8.83e-03 6.03e-03f  1
 371r 2.8357413e+04 9.15e+04 5.34e+03  -6.7 6.98e+02  -1.2 3.52e-02 4.60e-02f  1
 372r 2.8357413e+04 9.15e+04 5.34e+03  -6.7 2.06e+03  -1.7 6.19e-06 3.44e-09f  1
 373r 2.8399832e+04 9.15e+04 5.42e+03  -6.7 7.74e+02  -1.3 5.49e-02 1.62e-02f  1
 374r 2.8399832e+04 9.15e+04 5.42e+03  -6.7 2.90e+02  -0.9 2.44e-08 9.81e-09f  1
 375r 2.8508950e+04 9.14e+04 5.24e+03  -6.7 8.43e+02  -1.3 1.28e-02 3.66e-02f  1
 376r 2.8523344e+04 9.14e+04 5.17e+03  -6.7 3.17e+02  -0.9 1.40e-02 1.29e-02f  1
 377r 2.8585536e+04 9.14e+04 5.08e+03  -6.7 9.47e+02  -1.4 1.12e-07 1.86e-02f  1
 378r 2.8585536e+04 9.14e+04 4.93e+03  -6.7 3.56e+02  -1.0 4.57e-02 1.12e-08f  1
 379r 2.8590262e+04 9.14e+04 4.93e+03  -6.7 1.06e+03  -1.4 8.77e-09 1.30e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 2.8590262e+04 9.14e+04

 634  1.5832366e+05 5.86e+02 8.84e+05  -1.0 2.42e+03  -1.9 2.69e-02 7.53e-03h  1
 635  1.5832796e+05 5.85e+02 1.10e+06  -1.0 2.22e+03  -1.5 5.72e-02 1.12e-03h  1
 636  1.5835998e+05 5.80e+02 1.17e+06  -1.0 2.25e+03  -2.0 1.14e-01 9.09e-03h  1
 637  1.5837602e+05 5.78e+02 1.38e+06  -1.0 2.11e+03  -1.5 5.94e-02 4.30e-03h  1
 638  1.5843938e+05 5.68e+02 1.35e+06  -1.0 2.26e+03  -2.0 7.11e-03 1.70e-02h  1
 639  1.5844125e+05 5.67e+02 1.55e+06  -1.0 2.10e+03  -1.6 6.16e-02 5.06e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 640  1.5849665e+05 5.59e+02 1.55e+06  -1.0 2.23e+03  -2.1 1.07e-01 1.52e-02h  1
 641  1.5850966e+05 5.57e+02 2.41e+06  -1.0 2.11e+03  -1.6 2.84e-01 3.65e-03h  1
 642  1.5857784e+05 5.44e+02 2.27e+06  -1.0 2.22e+03  -2.1 1.36e-01 2.32e-02h  1
 643  1.5859706e+05 5.40e+02 3.13e+06  -1.0 2.02e+03  -1.7 3.66e-01 6.51e-03h  1
 644  1.5863318e+05 5.33e+02 3.10e+06  -1.0 2.01e+03  -2.2 8.71e-03 1.42e-02h  1
 645  1.5863834e+05 5.32e+02

 761  1.4150693e+05 4.92e+01 1.02e+06  -1.0 3.70e+04    -  2.26e-02 1.86e-04f  1
 762  1.3853527e+05 5.03e+01 9.34e+05  -1.0 3.75e+04    -  4.59e-04 2.39e-02f  1
 763  1.3571286e+05 5.11e+01 8.81e+05  -1.0 3.65e+04    -  8.80e-03 2.34e-02f  1
 764  1.3258041e+05 5.21e+01 8.37e+05  -1.0 3.56e+04    -  1.41e-02 2.67e-02f  1
 765  1.3230332e+05 5.19e+01 8.36e+05  -1.0 3.03e+04    -  1.31e-03 3.10e-03f  1
 766  1.3227030e+05 5.19e+01 9.05e+05  -1.0 3.43e+04    -  4.77e-02 2.96e-04f  1
 767  1.2958188e+05 5.27e+01 8.66e+05  -1.0 3.38e+04    -  8.67e-03 2.42e-02f  1
 768  1.2916940e+05 5.25e+01 8.67e+05  -1.0 2.88e+04    -  4.50e-04 4.77e-03f  1
 769  1.2911783e+05 5.24e+01 8.64e+05  -1.0 3.06e+04    -  8.72e-03 5.45e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 770  1.2844075e+05 5.22e+01 8.66e+05  -1.0 2.73e+04    -  4.04e-03 8.79e-03f  1
 771  1.2045726e+05 7.00e+01 9.53e+05  -1.0 2.65e+04    -  1.56e-02 1.04e-01f  1
 772  1.1951380e+05 6.85e+01

...

1281  3.6617325e+02 6.39e-03 3.73e+01  -5.7 3.89e+00  -0.4 8.09e-01 8.04e-02f  1
1282  3.6469827e+02 5.95e-03 3.54e+01  -5.7 1.50e+00   0.0 1.00e+00 6.75e-02f  1
1283  3.6215494e+02 5.70e-03 3.39e+01  -5.7 4.15e+00  -0.5 1.34e-02 4.20e-02f  1
1284  3.6117649e+02 5.46e-03 3.61e+01  -5.7 1.57e+00  -0.0 1.00e+00 4.27e-02f  1
1285  3.5839246e+02 5.22e-03 3.47e+01  -5.7 4.24e+00  -0.5 1.06e-02 4.34e-02f  1
1286  3.5828827e+02 5.20e-03 3.57e+01  -5.7 1.65e+00  -0.1 3.72e-01 4.31e-03f  1
1287  3.5589788e+02 5.02e-03 3.45e+01  -5.7 4.70e+00  -0.6 2.53e-02 3.40e-02f  1
1288  3.5561315e+02 4.97e-03 3.79e+01  -5.7 1.84e+00  -0.1 1.00e+00 1.07e-02f  1
1289  3.5387986e+02 4.86e-03 3.69e+01  -5.7 5.21e+00  -0.6 3.28e-02 2.24e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1290  3.5215323e+02 4.57e-03 3.35e+01  -5.7 2.06e+00  -0.2 6.36e-01 5.97e-02f  1
1291  3.5052470e+02 4.48e-03 3.26e+01  -5.7 5.75e+00  -0.7 3.25e-02 1.89e-02f  1
1292  3.4798209e+02 4.40e-03

1401  1.8452852e+02 5.52e-03 2.32e+03  -5.7 2.47e-01   0.1 1.00e+00 2.42e-03h  1
1402  1.8356925e+02 4.92e-03 2.39e+03  -5.7 7.58e-01  -0.4 6.83e-01 2.83e-01f  1
1403  1.8318437e+02 4.27e-03 1.95e+03  -5.7 2.88e-01   0.1 5.18e-01 3.19e-01h  1
1404  1.8298764e+02 2.41e-03 1.54e+03  -5.7 9.76e-02   0.5 1.00e+00 4.57e-01h  1
1405  1.8255120e+02 2.25e-03 7.01e+02  -5.7 2.95e-01   0.0 1.50e-01 3.29e-01h  1
1406  1.8245315e+02 1.82e-03 1.78e+03  -5.7 1.10e-01   0.4 1.00e+00 2.00e-01h  1
1407  1.8230059e+02 1.70e-03 1.70e+03  -5.7 1.15e+00  -0.0 3.06e-01 9.22e-02h  1
1408  1.8192114e+02 1.29e-03 9.02e+02  -5.7 1.19e-01   0.4 1.00e+00 7.08e-01h  1
1409  1.8101325e+02 8.77e-03 5.45e+02  -5.7 3.55e-01  -0.1 3.94e-01 5.65e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1410  1.8083614e+02 6.39e-03 2.80e+03  -5.7 2.69e-01   0.3 1.52e-01 3.71e-01h  1
1411  1.8072848e+02 3.00e-03 2.30e+03  -5.7 8.03e-02   0.8 1.00e+00 5.60e-01h  1
1412  1.8035476e+02 3.30e-03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1520  8.0311887e+01 8.95e-02 1.45e+03  -5.7 2.11e+01  -2.5 9.06e-03 6.30e-03f  1
1521  8.0311884e+01 8.95e-02 1.46e+03  -5.7 9.68e-01  -0.2 1.75e-03 4.18e-07h  2
1522  8.0311894e+01 8.95e-02 1.32e+03  -5.7 7.78e+01  -0.7 2.74e-02 2.04e-06f  7
1523  8.0160225e+01 7.70e-02 1.09e+03  -5.7 1.40e+00  -1.2 2.35e-01 1.39e-01h  1
1524  7.9852040e+01 7.06e-02 8.76e+02  -5.7 3.67e+00  -1.7 3.04e-01 8.38e-02h  1
1525  7.9841634e+01 7.05e-02 9.18e+02  -5.7 1.10e+01  -2.1 5.89e-02 9.35e-04h  1
1526  7.9199673e+01 6.92e-02 1.11e+03  -5.7 3.07e+01  -2.6 4.82e-02 1.94e-02f  1
1527  7.8040979e+01 6.83e-02 1.10e+03  -5.7 8.78e+01  -3.1 4.09e-03 1.27e-02f  1
1528  7.8040973e+01 6.83e-02 1.20e+03  -5.7 6.82e-01   0.0 3.85e-02 1.38e-06h  2
1529  7.8040972e+01 6.83e-02 8.16e+02  -5.7 3.16e-01  -0.4 2.16e-01 1.36e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1530  7.8002208e+01 6.18e-02

1634  7.3910988e+01 2.21e-03 2.02e-02  -5.7 4.46e-03   0.6 1.00e+00 1.00e+00f  1
1635  7.3910489e+01 5.34e-02 3.70e-01  -5.7 1.95e-02   0.1 1.00e+00 1.00e+00h  1
1636  7.3910016e+01 1.03e-02 7.73e-02  -5.7 8.53e-03   0.6 1.00e+00 1.00e+00h  1
1637  7.3907883e+01 3.07e-01 2.23e+00  -5.7 4.59e-02   0.1 1.00e+00 1.00e+00h  1
1638  7.3905167e+01 7.34e-02 5.86e-01  -5.7 2.14e-02   0.5 1.00e+00 1.00e+00h  1
1639  7.3900159e+01 3.86e-01 6.16e+02  -5.7 1.76e-01   0.0 6.45e-01 2.58e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1640  7.3900306e+01 3.12e-01 7.16e+01  -5.7 1.45e-02   0.5 1.00e+00 1.95e-01h  1
1641  7.3898785e+01 4.62e-03 7.15e-02  -5.7 6.24e-03  -0.0 1.00e+00 1.00e+00h  1
1642  7.3898625e+01 6.57e-04 2.35e-02  -5.7 2.54e-03   0.4 1.00e+00 1.00e+00h  1
1643  7.3898385e+01 9.07e-02 2.22e+00  -5.7 2.14e-02  -0.1 1.00e+00 1.00e+00h  1
1644  7.3897719e+01 2.30e-02 5.58e-01  -5.7 1.13e-02   0.4 1.00e+00 1.00e+00h  1
1645  7.3895299e+01 4.84e-01

1749r 7.3443443e+01 8.20e-01 1.00e+03  -0.1 0.00e+00   1.1 0.00e+00 4.67e-07R  2
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1750r 7.3443443e+01 8.20e-01 1.00e+03  -0.1 1.03e+10    -  1.84e-10 1.09e-10f  1
1751r 7.3505271e+01 1.43e+00 1.00e+03  -0.1 1.52e+06    -  1.03e-04 5.44e-05f  1
1752r 7.3887900e+01 2.02e+01 1.01e+03  -0.1 4.57e+05    -  1.26e-04 6.14e-04f  1
1753r 7.4328977e+01 2.00e+01 9.95e+02  -0.1 7.09e+04    -  7.01e-03 1.22e-03f  1
1754r 7.5742776e+01 3.34e+01 9.83e+02  -0.1 1.91e+04    -  1.16e-02 1.07e-02f  1
1755r 7.8718144e+01 3.86e+01 9.53e+02  -0.1 1.05e+04    -  3.12e-02 2.73e-02f  1
1756r 8.6926369e+01 5.09e+01 8.90e+02  -0.1 2.46e+03    -  6.29e-02 7.92e-02f  1
1757r 8.8653774e+01 8.73e+01 8.78e+02  -0.1 4.12e+02  -4.0 1.32e-02 1.27e-02f  1
1758r 8.9388479e+01 9.48e+01 8.69e+02  -0.1 4.31e+03    -  1.01e-02 5.55e-03f  1
1759r 9.2422821e+01 1.01e+02 8.07e+02  -0.1 9.38e+02    -  7.14e-02 3.52e-02f  1
iter    objective    inf_pr 

In [13]:
solve_pyomo(para_truth[9])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 6.2210937e+03 9.84e+04 2.87e+03  -3.0 3.85e+01  -0.4 5.16e-03 2.59e-02f  1
 188r 6.3101650e+03 9.83e+04 2.26e+03  -3.0 1.06e+02  -0.9 6.99e-02 2.31e-01f  1
 189r 6.3102725e+03 9.83e+04 2.54e+03  -3.0 1.05e+03    -  8.19e-03 2.25e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 6.3671256e+03 9.83e+04 3.69e+03  -3.0 1.05e+03    -  4.59e-02 1.19e-02f  1
 191r 6.5236756e+03 9.83e+04 5.39e+03  -3.0 1.04e+03    -  8.86e-02 3.32e-02f  1
 192r 6.5967394e+03 9.83e+04 4.83e+03  -3.0 1.00e+03    -  5.10e-04 1.60e-02f  1
 193r 6.6203476e+03 9.83e+04 6.21e+03  -3.0 9.86e+02    -  5.12e-02 5.26e-03f  1
 194r 6.7234016e+03 9.82e+04 5.46e+03  -3.0 9.80e+02    -  2.08e-03 2.31e-02f  1
 195r 6.8413861e+03 9.82e+04 4.62e+03  -3.0 2.29e+02  -1.4 1.36e-02 1.35e-01f  1
 196r 6.9281553e+03 9.82e+04 3.99e+03  -3.0 4.35e+02  -1.8 4.42e-03 4.83e-02f  1
 197r 6.9480161e+03 9.82e+04 3.89e+03  -3.0 1.37e+03    -  1.79e-03 4.77e-03f  1
 198r 6.9872656e+03 9.82e+04

 370r 3.0464117e+04 9.25e+04 4.69e+04  -4.5 1.68e+03  -1.6 1.93e-04 4.04e-06f  1
 371r 3.0489301e+04 9.25e+04 4.50e+04  -4.5 3.26e+02  -1.2 4.02e-02 2.19e-02f  1
 372r 3.0489308e+04 9.25e+04 4.31e+04  -4.5 1.27e+02  -0.8 3.88e-02 1.50e-05f  1
 373r 3.0584613e+04 9.24e+04 4.33e+04  -4.5 3.63e+02  -1.3 1.81e-05 7.37e-02f  1
 374r 3.0614855e+04 9.24e+04 4.15e+04  -4.5 1.41e+02  -0.8 4.10e-02 6.08e-02f  1
 375r 3.0744746e+04 9.24e+04 4.16e+04  -4.5 4.07e+02  -1.3 5.68e-06 9.26e-02f  1
 376r 3.0744755e+04 9.24e+04 4.05e+04  -4.5 1.59e+02  -0.9 2.58e-02 1.67e-05f  1
 377r 3.0907303e+04 9.24e+04 3.88e+04  -4.5 4.51e+02  -1.4 4.25e-02 1.05e-01f  1
 378r 3.0907315e+04 9.24e+04 3.74e+04  -4.5 1.77e+02  -0.9 3.37e-02 2.00e-05f  1
 379r 3.0983040e+04 9.23e+04 3.73e+04  -4.5 4.98e+02  -1.4 1.79e-03 4.38e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 3.0983065e+04 9.23e+04 3.65e+04  -4.5 1.97e+02  -1.0 2.09e-02 3.72e-05f  1
 381r 3.1090570e+04 9.23e+04

 684  2.2214045e+05 5.47e+03 1.98e+06  -1.0 5.47e+03  -0.7 3.01e-03 9.61e-05h  1
 685  2.2215614e+05 5.47e+03 1.98e+06  -1.0 5.47e+03  -1.2 1.43e-03 1.47e-03h  1
 686  2.2215697e+05 5.46e+03 2.30e+06  -1.0 5.47e+03  -0.8 1.37e-02 7.25e-05h  1
 687  2.2216929e+05 5.46e+03 2.29e+06  -1.0 5.46e+03  -1.2 1.04e-03 1.26e-03h  1
 688  2.2217026e+05 5.46e+03 2.35e+06  -1.0 5.46e+03  -0.8 2.52e-03 9.15e-05h  1
 689  2.2218238e+05 5.45e+03 2.37e+06  -1.0 5.46e+03  -1.3 4.93e-03 1.19e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 690  2.2218456e+05 5.45e+03 2.41e+06  -1.0 5.45e+03  -0.9 1.70e-03 1.73e-04h  1
 691  2.2220245e+05 5.44e+03 2.51e+06  -1.0 5.45e+03  -1.3 1.01e-02 1.19e-03h  1
 692  2.2222841e+05 5.43e+03 2.52e+06  -1.0 6.04e+03  -1.8 5.51e-03 1.74e-03h  1
 693  2.2223389e+05 5.43e+03 3.28e+06  -1.0 5.43e+03  -1.4 5.96e-02 3.64e-04h  1
 694  2.2229933e+05 5.41e+03 3.27e+06  -1.0 5.43e+03  -1.9 4.98e-03 4.54e-03h  1
 695  2.2232206e+05 5.40e+03

 809  2.1648986e+05 1.75e+03 1.06e+06  -1.0 2.88e+03  -2.8 7.42e-04 7.68e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 810  2.1652825e+05 1.73e+03 9.36e+05  -1.0 2.73e+03  -3.3 9.22e-02 9.93e-03h  1
 811  2.1702963e+05 1.36e+03 1.06e+06  -1.0 2.76e+03  -3.8 1.13e-03 2.17e-01h  1
 812  2.1722524e+05 1.25e+03 1.01e+06  -1.0 4.33e+03  -2.4 4.11e-04 8.04e-02h  1
 813  2.1723254e+05 1.24e+03 8.57e+05  -1.0 2.18e+03  -2.9 1.27e-01 3.22e-03h  1
 814  2.1723326e+05 1.24e+03 8.57e+05  -1.0 1.92e+03  -3.4 5.30e-04 9.97e-04h  1
 815  2.1721770e+05 1.23e+03 8.57e+05  -1.0 1.75e+03  -3.9 2.63e-03 6.59e-03f  1
 816  2.1721775e+05 1.23e+03 8.33e+05  -1.0 2.18e+03  -3.4 1.83e-02 2.61e-04h  1
 817  2.1711162e+05 1.20e+03 8.46e+05  -1.0 2.30e+03  -3.9 1.58e-03 2.45e-02f  1
 818  2.1711212e+05 1.20e+03 7.34e+05  -1.0 2.37e+03  -3.5 8.70e-02 5.99e-03h  1
 819  2.1711792e+05 1.18e+03 7.39e+05  -1.0 2.50e+03  -4.0 1.58e-03 1.29e-02h  1
iter    objective    inf_pr 

...

1924  9.9027258e+01 1.46e-11 3.09e-03  -5.7 1.03e-04   1.5 1.00e+00 1.00e+00h  1
1925  9.9027247e+01 1.46e-11 2.20e-03  -5.7 2.19e-04   1.0 1.00e+00 1.00e+00h  1
1926  9.9027215e+01 1.46e-11 1.93e-03  -5.7 5.76e-04   0.5 1.00e+00 1.00e+00f  1
1927  9.9027125e+01 1.46e-11 1.82e-03  -5.7 1.63e-03   0.0 1.00e+00 1.00e+00f  1
1928  9.9026969e+01 1.49e-11 1.88e+00  -5.7 4.47e-03  -0.4 1.00e+00 6.31e-01f  1
1929  9.9026708e+01 4.37e-11 5.80e+00  -5.7 1.15e-02  -0.9 1.00e+00 4.14e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1930  9.9024932e+01 1.12e-09 1.36e-03  -5.7 3.29e-02  -1.4 1.00e+00 1.00e+00f  1
1931  9.9022845e+01 2.07e-09 1.33e+00  -5.7 9.81e-02  -1.9 1.00e+00 3.95e-01f  1
1932  9.9017408e+01 9.67e-09 4.46e+00  -5.7 2.94e-01  -2.3 1.00e+00 3.47e-01f  1
1933  9.9014786e+01 9.81e-09 1.88e+01  -5.7 8.66e-01  -2.8 1.00e+00 5.73e-02f  1
1934  9.8972705e+01 1.27e-04 3.94e+01  -5.7 2.51e+00  -3.3 1.00e+00 3.19e-01f  1
1935  9.8915888e+01 3.55e-04

2052  8.0290509e+01 1.90e-01 5.73e+01  -5.7 2.91e-02   0.7 1.00e+00 4.96e-01h  1
2053  8.0290512e+01 1.27e-01 9.89e+01  -5.7 3.81e-02   0.2 7.48e-01 3.35e-01h  1
2054  8.0259334e+01 1.26e-01 9.46e+01  -5.7 2.43e+02    -  1.40e-03 2.65e-02h  1
2055  8.0259342e+01 1.23e-01 1.52e+02  -5.7 1.01e-02  -0.3 1.00e+00 2.43e-02h  1
2056  8.0260072e+01 8.73e-11 2.08e-03  -5.7 1.15e-02  -0.7 1.00e+00 1.00e+00h  1
2057  8.0129965e+01 5.59e-02 1.01e+02  -5.7 7.83e+04    -  8.12e-05 3.43e-04f  1
2058  8.0129985e+01 5.24e-02 1.32e+01  -5.7 3.78e-03  -1.2 1.00e+00 6.25e-02h  5
2059  8.0076525e+01 7.33e-02 4.69e+01  -5.7 8.45e+03    -  7.93e-04 4.49e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2060  7.9865126e+01 4.32e-01 3.72e+01  -5.7 1.68e+03    -  3.83e-02 2.78e-02f  1
2061  7.9062864e+01 1.10e+01 1.38e+02  -5.7 8.32e+02    -  2.46e-01 2.17e-01f  1
2062  7.9063808e+01 1.09e+01 1.31e+02  -5.7 1.73e-01   0.1 2.27e-02 6.02e-03h  1
2063  7.9030945e+01 1.08e+01

2186r 7.7523329e+01 1.80e+01 1.00e+03   1.3 0.00e+00    -  0.00e+00 1.85e-10R 13
2187r 7.7523329e+01 1.80e+01 1.11e+04   1.3 1.00e+09    -  4.75e-10 1.24e-13f  1
2188r 7.7709030e+01 1.81e+01 1.11e+04   1.3 1.62e+07    -  3.66e-08 7.15e-06f  1
2189r 7.8739689e+01 1.90e+01 1.11e+04   1.3 1.01e+06    -  2.47e-04 1.74e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2190r 8.1754319e+01 5.63e+00 1.09e+04   1.3 1.39e+05    -  8.70e-03 1.23e-03f  1
2191r 9.0967159e+01 2.28e+01 1.09e+04   1.3 2.62e+04    -  2.06e-03 1.12e-02f  1
2192r 9.6165209e+01 2.71e+01 9.80e+03   1.3 2.15e+00   2.0 1.04e-01 5.83e-02f  1
2193r 1.1893363e+02 8.25e+01 8.52e+03   1.3 5.37e+03    -  1.10e-01 1.65e-01f  1
2194r 1.3302237e+02 1.20e+02 7.79e+03   1.3 9.93e+02    -  6.44e-02 1.12e-01f  1
2195r 1.5653344e+02 1.30e+02 7.04e+02   1.3 1.47e+00   1.5 1.00e+00 6.69e-01f  1
2196r 1.5214522e+02 5.94e+01 6.21e+02   0.6 6.11e+02    -  3.80e-01 5.17e-01f  1
2197r 1.4763906e+02 6.81e-02

2351  7.7547528e+01 3.44e-06 2.24e-02  -8.6 1.96e-01  -4.0 1.00e+00 8.35e-01f  1
2352  7.7547519e+01 3.39e-06 1.39e-01  -8.6 5.88e-01  -4.5 3.78e-01 7.05e-02f  1
2353  7.7547493e+01 6.27e-06 9.57e-01  -8.6 1.52e+00  -5.0 1.00e+00 7.66e-02f  1
2354  7.7547478e+01 7.32e-06 1.54e+00  -8.6 3.90e+00  -5.5 3.65e-01 1.78e-02f  1
2355  7.7547455e+01 1.01e-05 6.30e+00  -8.6 9.85e+00  -6.0 9.27e-01 1.08e-02f  1
2356  7.7546236e+01 8.06e-03 2.60e+00  -8.6 2.72e+01  -6.4 1.14e-02 2.06e-01f  1
2357  7.7545048e+01 1.50e-02 4.23e+01  -8.6 7.09e+01  -6.9 1.00e+00 8.03e-02f  1
2358  7.7543829e+01 2.24e-02 4.13e+01  -8.6 1.56e+02  -7.4 4.46e-02 3.60e-02h  1
2359  7.7542953e+01 2.61e-02 6.32e+01  -8.6 2.99e+02  -7.9 1.38e-01 1.34e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2360  7.7542953e+01 2.61e-02 1.18e+01  -8.6 1.87e-02  -0.0 1.00e+00 4.31e-04h  1
2361  7.7542953e+01 2.57e-02 5.47e-02  -8.6 1.42e-04  -0.5 1.00e+00 1.56e-02h  7
2362  7.7542971e+01 1.46e-11

2465  7.7527697e+01 1.46e-11 2.00e-05  -8.6 3.05e-05  -0.2 1.00e+00 1.00e+00h  1
2466  7.7527697e+01 1.46e-11 3.76e-06  -8.6 1.72e-05  -0.7 1.00e+00 1.00e+00h  1
2467  7.7527697e+01 1.46e-11 3.72e-06  -8.6 5.10e-05  -1.1 1.00e+00 1.00e+00h  1
2468  7.7527697e+01 1.46e-11 3.65e-06  -8.6 1.50e-04  -1.6 1.00e+00 1.00e+00h  1
2469  7.7527696e+01 1.46e-11 3.55e-06  -8.6 4.38e-04  -2.1 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2470  7.7527696e+01 1.46e-11 3.47e-06  -8.6 1.28e-03  -2.6 1.00e+00 1.00e+00h  1
2471  7.7527696e+01 2.91e-11 3.44e-06  -8.6 3.82e-03  -3.0 1.00e+00 1.00e+00h  1
2472  7.7527693e+01 1.33e-10 3.43e-06  -8.6 1.14e-02  -3.5 1.00e+00 1.00e+00h  1
2473  7.7527689e+01 1.09e-09 1.54e-02  -8.6 3.43e-02  -4.0 1.00e+00 7.10e-01h  1
2474  7.7527687e+01 1.19e-09 1.46e-01  -8.6 1.08e-01  -4.5 1.00e+00 1.10e-01f  1
2475  7.7527635e+01 1.22e-05 2.92e-02  -8.6 2.86e-01  -5.0 1.00e+00 9.39e-01f  1
2476  7.7527627e+01 1.18e-05